# Necessary to load the OxidizeME docker image
### Old instructions doi: 10.1073/pnas.1905039116 / https://github.com/SBRG/oxidizeme
### Updated installation instructions on https://github.com/QCSB/StressME?tab=readme-ov-file

In [1]:
import cobrame
import ecolime
from cobra import Reaction
from cobra import Metabolite
from qminospy.me1 import ME_NLP1

import numpy as np
from numpy.random import normal, uniform
import scipy
import math
import pickle
import pandas as pd
import argparse
import time
import re
from cobrame.util import dogma, building
import cobrame.util.building

/home/.local/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/solvers/__init__.py:63: UserWarning: No LP solvers found
  warn("No LP solvers found")
/home/.local/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/.local/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


In [2]:
!ls

fort.11               make_oxidizeme.ipynb  raw_tpm_data.csv
fort.9                me_models             solve_demo.ipynb
html_files            phenotype.csv         spont_ros_rxns.csv


In [3]:
def show_escher_map(me, solution=None):
    import escher
    view = escher.Builder("iJO1366.Central metabolism")
    view.reaction_data = me.get_metabolic_flux(solution=solution)
    return view

In [4]:
#============================================================
# Build the ME model.
# Then, make the OxidizeME model from the fresh ME model.
#============================================================
from oxidizeme.model import StressME
from qminospy.me1 import ME_NLP1
from ecolime import build_me_model
import cloudpickle
import pickle
import argparse
import re
import os

#--------------------------------------------------------
# Load the ME model
with open('/home/meuser/me_models/iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)

#--------------------------------------------------------
# Build StressME model
print('#--------------------------------------------------------')
print("Making stressME")
solver = ME_NLP1(me)
stress = StressME(solver)

# Make stressME
stress.make_stressme()

# Force demetallation and mismetallation via coupling
stress.force_demetallation(csense='G')
stress.force_mismetallation(csense='G')

# Force Fe-S cluster damage
stress.force_fes_damage(csense='G')

# Need to open some bounds
print("Opening some bounds")
rxns_open = ['SPODM', 'SPODMpp', 'AHPRED', 'CAT']
for rid_open in rxns_open:
    try:
        data = me.stoichiometric_data.get_by_id(rid_open)
        for rxn in data.parent_reactions:
            rxn.upper_bound = 1000.
    except:
        pass

# Ensure detoxification proteins can be expressed
print("Allowing detox fluxes")
detox_prots = ['b3908', 'b1656', 'b1646', ] #'b0606', 'b0605' not in model

for bnum in detox_prots:
    rid = 'translation_'+bnum
    if rid in me.reactions:
        rxn = me.reactions.get_by_id(rid)
        rxn.upper_bound = 1000.
    else:
        print('%s not in model. Skipping.'%rid)

#--------------------------------------------------------
Making stressME
Created <TranslatedGene protein_b0605 at 0x7f2b5b587630> in <ComplexFormation formation_CPLX0-245 at 0x7f2b5b587518>
Created <TranslatedGene protein_b0606 at 0x7f2b5b587438> in <ComplexFormation formation_CPLX0-245 at 0x7f2b5b587518>
('mod_mn2_c', 'already in model.')
('mod_zn2_c', 'already in model.')
('mod_cobalt2_c', 'already in model.')
Created <TranslatedGene protein_b2962 at 0x7f2b5b536780> in <ComplexFormation formation_yggX at 0x7f2b5b5365f8>
Created <TranslatedGene protein_b4209 at 0x7f2b5b536b38> in <ComplexFormation formation_ytfE at 0x7f2b5b5369b0>
Created <TranslatedGene protein_b3662 at 0x7f2b5b4eb0b8> in <ComplexFormation formation_YICM-MONOMER at 0x7f2b5b561f60>
Created <TranslatedGene protein_b0812 at 0x7f2b5b4eb588> in <ComplexFormation formation_CPLX0-1521 at 0x7f2b5b4eb470>
Created <TranslatedGene protein_b3961 at 0x7f2b5b4ebb00> in <ComplexFormation formation_TF_OxyR at 0x7f2b5b4eb9b0>
Create

In [5]:
#growth media
ex_rxns = [r for r in me.reactions if r.id.startswith('EX_')]
open_ex_rxns = [r for r in me.reactions if r.id.startswith('EX_') and r.lower_bound<0]

In [6]:
#define helper functions for expression constraint
def create_reaction_sum_of_expression(me,group_name,gene_list):

    try:
        rxn_group_sum = me.reactions.get_by_id(group_name)
    except:
        rxn_group_sum = Reaction(group_name)
        me.add_reaction(rxn_group_sum)

        cons = Metabolite('sum_'+group_name)
        cons._constraint_sense = 'E'
        cons._bound = 0
        me.add_metabolites(cons)

        for gene in gene_list:
            try:
                rxn = me.reactions.get_by_id('translation_'+gene)
                met = me.metabolites.get_by_id('protein_'+gene)
                rxn.add_metabolites({cons: 1.*met.formula_weight}, combine=True)
            except:
                print (gene,'Not modeled')
        rxn_group_sum.add_metabolites({cons: -1})


def set_expression_ratio(me,group_name1,group_name2,ratio):

    try:
        rxn1 = me.reactions.get_by_id(group_name1)
        rxn1._metabolites[me.metabolites.get_by_id(group_name1+'_'+group_name2+'_ratio')]=1.
        rxn2 = me.reactions.get_by_id(group_name2)
        rxn2._metabolites[me.metabolites.get_by_id(group_name1+'_'+group_name2+'_ratio')]=-1.*ratio
    except:
        cons = Metabolite(group_name1+'_'+group_name2+'_ratio')
        cons._constraint_sense = 'G'
        cons._bound = 0
        me.add_metabolites(cons)

        rxn_group1_sum = me.reactions.get_by_id(group_name1)
        rxn_group1_sum.add_metabolites({cons: 1.})
        rxn_group2_sum = me.reactions.get_by_id(group_name2)
        rxn_group2_sum.add_metabolites({cons:-1.*ratio})

def get_enzyme_pathway_ratio(mf_enzyme,mf_pathway):
    return [mf_enzyme[i]/mf_pathway[i] for i in range(0,len(mf_pathway))]

MW_dict={}
protein_length={}
for gene in me.metabolites.query(re.compile('RNA_b[0-9]')):
    prot_id=gene.id.replace('RNA','protein')
    try:
        met = me.metabolites.get_by_id(prot_id)
        MW_dict[prot_id.replace('protein_','')] = met.formula_weight
        protein_length[prot_id.replace('protein_','')] = len(met.amino_acid_sequence)
    except:
        continue # this is a RNA gene
model_genes=sorted(MW_dict.keys())
MW_dict['dummy']=33710.96300000001 # average molecular weight for E. coli proteins
mass_glc=0.1800634

def get_total_protein(sol):
    total_protein = 0
    mf_dict={}
    if type(sol) != type(None):
        for a,b in sol.x_dict.items():
            if a.startswith('translation_') and b > 1e-20:
                gene = a.replace('translation_','')
                flux = sol.x_dict['translation_'+gene]
                mw=MW_dict[gene]
                total_protein += flux*mw
                mf_dict[gene] = flux*mw
        return total_protein,mf_dict
    else:
        print ("No feasible solution!")
        return -1

def get_proteome_sector_fraction(sol,proteome_sector):
    total_protein,mf_dict=get_total_protein(sol)
    mf=0
    for gene in proteome_sector:
        try:
            flux=sol.x_dict['translation_'+gene]
            mf+=MW_dict[gene]*flux
        except:
            # print(gene, 'Not in model')
            pass
    mf=mf/total_protein
    return mf

In [7]:
def get_sum(sol, gene_list):
    
    gene_rxn = []
    for i in gene_list:
        gene_rxn.append(i[-5:])
        
    total = 0
    for i in gene_rxn:
        try:
            get_total_protein(sol)[1][i]
            total = total + get_total_protein(sol)[1][i]
        except:
            total = total
    return total/get_total_protein(sol)[0]

In [8]:
def get_sum_damage(sol):
    
    damage_rxns = {'damage_SUCC-DEHASE_mod_3fe4s_mod_fad_mod_2fe2s_mod_4fe4s_h2o2':64422,
    'damage_SUCC-DEHASE_mod_3fe4s_mod_fad_mod_2fe2s_mod_4fe4s_o2s':64422,
    'damage_CPLX0-7760_mod_4fe4s_h2o2':93498,
    'damage_CPLX0-7760_mod_4fe4s_o2s':93498,
    'damage_CPLX0-7761_mod_4fe4s_h2o2':93498,
    'damage_CPLX0-7761_mod_4fe4s_o2s':93498,
    'damage_DIHYDROXYACIDDEHYDRAT-CPLX_mod_4fe4s_h2o2':65532,
    'damage_DIHYDROXYACIDDEHYDRAT-CPLX_mod_4fe4s_o2s':65532,
    'damage_LTARTDEHYDRA-CPLX_mod_4fe4s_h2o2':22679,
    'damage_LTARTDEHYDRA-CPLX_mod_4fe4s_o2s':22679,
    'damage_NADH-DHI-CPLX_mod_2fe2s_mod_4fe4s_mod_fmn_h2o2':16457,
    'damage_NADH-DHI-CPLX_mod_2fe2s_mod_4fe4s_mod_fmn_o2s':16457,
    'damage_CPLX0-7622_mod_4fe4s_h2o2':48522,
    'damage_CPLX0-7622_mod_4fe4s_o2s':48522,
    'damage_CPLX0-7719_mod_4fe4s_h2o2':38241,
    'damage_CPLX0-7719_mod_4fe4s_o2s':38241,
    'damage_NITRATREDUCTA-CPLX_mod_bmocogdp_mod_3fe4s_mod_4fe4s_h2o2':140490,
    'damage_NITRATREDUCTA-CPLX_mod_bmocogdp_mod_3fe4s_mod_4fe4s_o2s':140490,
    'damage_FUMARASE-A_mod_4fe4s_h2o2':60299,
    'damage_FUMARASE-A_mod_4fe4s_o2s':60299,
    'damage_LSERINEDEAM1-MONOMER_mod_4fe4s_h2o2':48907,
    'damage_LSERINEDEAM1-MONOMER_mod_4fe4s_o2s':48907,
    'damage_QueG_mono_mod_2:4fe4s_h2o2':43073,
    'damage_QueG_mono_mod_2:4fe4s_o2s':43073,
    'damage_CPLX0-250_FORMATEDEHYDROGH-MONOMER_mod_8:4fe4s_mod_2:2fe2s_mod_bmocogdp_mod_NiFeCoCN2_h2o2':34747,
    'damage_CPLX0-250_FORMATEDEHYDROGH-MONOMER_mod_8:4fe4s_mod_2:2fe2s_mod_bmocogdp_mod_NiFeCoCN2_o2s':34747,
    'damage_YdaO_dim_mod_4fe4s_h2o2':35561,
    'damage_YdaO_dim_mod_4fe4s_o2s':35561,
    'damage_MiaB_mono_mod_1:4fe4s_h2o2':53663,
    'damage_MiaB_mono_mod_1:4fe4s_o2s':53663,
    'damage_FUMARASE-B_mod_4fe4s_h2o2':60105,
    'damage_FUMARASE-B_mod_4fe4s_o2s':60105,
    'damage_LSERINEDEAM2-MONOMER_mod_4fe4s_h2o2':48753,
    'damage_LSERINEDEAM2-MONOMER_mod_4fe4s_o2s':48753,
    'damage_RumB_mono_mod_1:4fe4s_h2o2':41956,
    'damage_RumB_mono_mod_1:4fe4s_o2s':41956,
    'damage_NITRATREDUCTZ-CPLX_mod_bmocogdp_mod_2:pheme_mod_3:4fe4s_mod_1:3fe4s_h2o2':58558,
    'damage_NITRATREDUCTZ-CPLX_mod_bmocogdp_mod_2:pheme_mod_3:4fe4s_mod_1:3fe4s_o2s':58558,
    'damage_HEMN-MONOMER_mod_4fe4s_h2o2':52729,
    'damage_HEMN-MONOMER_mod_4fe4s_o2s':52729,
    'damage_RlmN_mono_mod_1:4fe4s_h2o2':43086,
    'damage_RlmN_mono_mod_1:4fe4s_o2s':43086,
    'damage_FORMHYDROGI-CPLX_mod_ni2_mod_4fe4s_h2o2':79374,
    'damage_FORMHYDROGI-CPLX_mod_ni2_mod_4fe4s_o2s':79374,
    'damage_RumA_mono_mod_1:4fe4s_h2o2':48052,
    'damage_RumA_mono_mod_1:4fe4s_o2s':48052,
    'damage_BIOTIN-SYN-CPLX_mod_4fe4s_mod_2fe2s_h2o2':38648,
    'damage_BIOTIN-SYN-CPLX_mod_4fe4s_mod_2fe2s_o2s':38648,
    'damage_SULFITE-REDUCT-CPLX_mod_4fe4s_mod_sheme_mod_fmn_mod_fad_h2o2':63998,
    'damage_SULFITE-REDUCT-CPLX_mod_4fe4s_mod_sheme_mod_fmn_mod_fad_o2s':63998,
    'damage_CPLX0-782_mod_2:4fe4s_h2o2':36072,
    'damage_CPLX0-782_mod_2:4fe4s_o2s':36072,
    'damage_FUMARATE-REDUCTASE_mod_1:pheme_mod_3fe4s_mod_4fe4s_mod_fad_mod_2fe2s_h2o2':65972,
    'damage_FUMARATE-REDUCTASE_mod_1:pheme_mod_3fe4s_mod_4fe4s_mod_fad_mod_2fe2s_o2s':65972,
    'damage_IscU_mod_1:4fe4s_h2o2':13849,
    'damage_IscU_mod_1:4fe4s_o2s':13849,               
    'damage_LSERINEDEAM1-MONOMER_mod_4fe4s_h2o2':48907,
    'damage_LSERINEDEAM1-MONOMER_mod_4fe4s_o2s':48907,
    'damage_QueG_mono_mod_2:4fe4s_h2o2':43073,
    'damage_QueG_mono_mod_2:4fe4s_o2s':43073,
    'damage_CPLX0-250_FORMATEDEHYDROGH-MONOMER_mod_8:4fe4s_mod_2:2fe2s_mod_bmocogdp_mod_NiFeCoCN2_h2o2':34747,
    'damage_CPLX0-250_FORMATEDEHYDROGH-MONOMER_mod_8:4fe4s_mod_2:2fe2s_mod_bmocogdp_mod_NiFeCoCN2_o2s':34747,
    'damage_YdaO_dim_mod_4fe4s_h2o2':35561,
    'damage_YdaO_dim_mod_4fe4s_o2s':35561,
    'damage_MiaB_mono_mod_1:4fe4s_h2o2':53663,
    'damage_MiaB_mono_mod_1:4fe4s_o2s':53663,
    'damage_FUMARASE-B_mod_4fe4s_h2o2':60105,
    'damage_FUMARASE-B_mod_4fe4s_o2s':60105,
    'damage_LSERINEDEAM2-MONOMER_mod_4fe4s_h2o2':48753,
    'damage_LSERINEDEAM2-MONOMER_mod_4fe4s_o2s':48753,
    'damage_RumB_mono_mod_1:4fe4s_h2o2':41956,
    'damage_RumB_mono_mod_1:4fe4s_o2s':41956,
    'damage_NITRATREDUCTZ-CPLX_mod_bmocogdp_mod_2:pheme_mod_3:4fe4s_mod_1:3fe4s_h2o2':58558,
    'damage_NITRATREDUCTZ-CPLX_mod_bmocogdp_mod_2:pheme_mod_3:4fe4s_mod_1:3fe4s_o2s':58558,
    'damage_HEMN-MONOMER_mod_4fe4s_h2o2':52729,
    'damage_HEMN-MONOMER_mod_4fe4s_o2s':52729,
    'damage_RlmN_mono_mod_1:4fe4s_h2o2':43086,
    'damage_RlmN_mono_mod_1:4fe4s_o2s':43086,
    'damage_FORMHYDROGI-CPLX_mod_ni2_mod_4fe4s_h2o2':79374,
    'damage_FORMHYDROGI-CPLX_mod_ni2_mod_4fe4s_o2s':79374,
    'damage_RumA_mono_mod_1:4fe4s_h2o2':48052,
    'damage_RumA_mono_mod_1:4fe4s_o2s':48052,
    'damage_BIOTIN-SYN-CPLX_mod_4fe4s_mod_2fe2s_h2o2':38648,
    'damage_BIOTIN-SYN-CPLX_mod_4fe4s_mod_2fe2s_o2s':38648,
    'damage_SULFITE-REDUCT-CPLX_mod_4fe4s_mod_sheme_mod_fmn_mod_fad_h2o2':63998,
    'damage_SULFITE-REDUCT-CPLX_mod_4fe4s_mod_sheme_mod_fmn_mod_fad_o2s':63998,
    'damage_CPLX0-782_mod_2:4fe4s_h2o2':36072,
    'damage_CPLX0-782_mod_2:4fe4s_o2s':36072,
    'damage_FUMARATE-REDUCTASE_mod_1:pheme_mod_3fe4s_mod_4fe4s_mod_fad_mod_2fe2s_h2o2':65972,
    'damage_FUMARATE-REDUCTASE_mod_1:pheme_mod_3fe4s_mod_4fe4s_mod_fad_mod_2fe2s_o2s':65972,
    'damage_IscU_mod_1:4fe4s_h2o2':13849,
    'damage_IscU_mod_1:4fe4s_o2s':13849}    
    
    total_mass = 0
    for i in damage_rxns.keys():
        flux = sol.x_dict[i]
        mass = flux*damage_rxns[i]
        total_mass = total_mass+mass
    return total_mass/get_total_protein(sol)[0]

In [9]:
#gene_lists
rproteins = ['b1480','b3065','b0023','b3316','b4202','b3311','b2609','b3165','b3307','b3298','b3342','b3297','b3321','b3230','b3306','b3341','b4200','b3303','b3296','b3314','b0169','b0911','b3851','b3986','b3985','b3299','b1717','b3703','b3636','b1089','b3936','b3302','b3312','b3637','b3185','b2185','b3309','b3318','b3315','b3186','b1716','b2606','b3304','b3294','b3313','b3301','b3310','b3231','b3983','b4203','b3305','b3308','b3319','b3320','b3317','b3984','b3855','b3854']
glycolysis = ['b4025','b0755','b3925','b2925','b2926','b1779','b2097','b1702','b3916','b3919','b3612','b2779','b1676','b1854','b0383','b4232','b1723']
PPP = ['b1850','b1851','b1852','b0767','b2029', 'b3386','b2914','b4090','b2465','b2935','b0008','b2464','b2935']
TCA = ['b1611','b1136','b1276','b0116','b0723','b0724','b0721','b0722','b0729','b0726','b0727','b1612','b0728','b0720','b4122','b1675','b3236','b1479','b0118','b2929']
# oxPPP = ['b1850','b1851','b1852','b0767','b2029']
# nonoxPPP = ['b3386','b2914','b4090','b2465','b2935','b0008','b2464','b2935']
list_resistance = ['b1732', 'b0871', 'b2464', 'b1897', 'b2131', 'b3073', 'b1896', 'b3447', 'b0789', 'b1478', 'b2097', 'b2130', 'b1480', 'b1927', 'b2661', 'b2128', 'b2129', 'b1197', 'b2662', 'b1519', 'b3588', 'b1440', 'b1646', 'b2741', 'b2663', 'b0837', 'b0186', 'b1441', 'b3496', 'b3012', 'b2465', 'b1519', 'b1682', 'b1681', 'b1683', 'b1680', 'b1679', 'b1684', 'b3517', 'b1493', 'b1492', 'b0485', 'b3907']
list_persistence = ['b1922', 'b3924', 'b1611', 'b1378', 'b1463', 'b3908', 'b1377', 'b0871', 'b3909', 'b1852', 'b2963', 'b0684', 'b1277', 'b3519', 'b2889', 'b2895', 'b1276', 'b1533', 'b0462', 'b0628', 'b2411', 'b0168', 'b0463', 'b0578', 'b0849', 'b0931', 'b0369', 'b0096', 'b0160', 'b2964', 'b3146', 'b0870', 'b2558', 'b1851', 'b0766', 'b0124', 'b3198', 'b3917', 'b3546', 'b0767', 'b3201', 'b3035', 'b2174', 'b0469', 'b3200', 'b3199', 'b1101', 'b1215', 'b0929', 'b3666', 'b0763', 'b0765', 'b0764', 'b1922', 'b0014', 'b4143', 'b4142', 'b0015', 'b2614', 'b3343', 'b1779', 'b1380', 'b1114', 'b4171', 'b3179', 'b4034', 'b3458', 'b3457', 'b3460', 'b3456', 'b3455', 'b3454', 'b0077', 'b0078', 'b3774', 'b3829', 'b1602', 'b1603', 'b2290', 'b0072', 'b0929', 'b0863', 'b0861', 'b1907', 'b0381', 'b1814', 'b4129', 'b3496', 'b1605', 'b3616', 'b2797', 'b3617', 'b2796', 'b1634', 'b1189', 'b1190', 'b2143', 'b3831', 'b4381', 'b4382', 'b2964', 'b0411', 'b3366', 'b3365']
proteome = ['b0001', 'b0002', 'b0003', 'b0004', 'b0005', 'b0006', 'b0007', 'b0008', 'b0009', 'b0010', 'b0011', 'b0013', 'b0014', 'b0015', 'b0016', 'b0018', 'b0019', 'b0020', 'b0021', 'b0022', 'b0023', 'b0024', 'b0025', 'b0026', 'b0027', 'b0028', 'b0029', 'b0030', 'b0031', 'b0032', 'b0033', 'b0034', 'b0035', 'b0036', 'b0037', 'b0038', 'b0039', 'b0040', 'b0041', 'b0042', 'b0043', 'b0044', 'b0045', 'b0046', 'b0047', 'b0048', 'b0049', 'b0050', 'b0051', 'b0052', 'b0053', 'b0054', 'b0055', 'b0058', 'b0059', 'b0060', 'b0061', 'b0062', 'b0063', 'b0064', 'b0065', 'b0066', 'b0067', 'b0068', 'b0069', 'b0070', 'b0071', 'b0072', 'b0073', 'b0074', 'b0075', 'b0076', 'b0077', 'b0078', 'b0080', 'b0081', 'b0082', 'b0083', 'b0084', 'b0085', 'b0086', 'b0087', 'b0088', 'b0089', 'b0090', 'b0091', 'b0092', 'b0093', 'b0094', 'b0095', 'b0096', 'b0097', 'b0098', 'b0099', 'b0101', 'b0102', 'b0103', 'b0104', 'b0106', 'b0107', 'b0108', 'b0109', 'b0110', 'b0111', 'b0112', 'b0113', 'b0114', 'b0115', 'b0116', 'b0117', 'b0118', 'b0119', 'b0120', 'b0121', 'b0122', 'b0123', 'b0124', 'b0125', 'b0126', 'b0127', 'b0128', 'b0129', 'b0130', 'b0131', 'b0132', 'b0133', 'b0134', 'b0135', 'b0136', 'b0137', 'b0138', 'b0139', 'b0140', 'b0141', 'b0142', 'b0143', 'b0144', 'b0145', 'b0146', 'b0147', 'b0148', 'b0149', 'b0150', 'b0151', 'b0152', 'b0153', 'b0154', 'b0155', 'b0156', 'b0157', 'b0158', 'b0159', 'b0160', 'b0161', 'b0162', 'b0163', 'b0164', 'b0166', 'b0167', 'b0168', 'b0169', 'b0170', 'b0171', 'b0172', 'b0173', 'b0174', 'b0175', 'b0176', 'b0177', 'b0178', 'b0179', 'b0180', 'b0181', 'b0182', 'b0183', 'b0184', 'b0185', 'b0186', 'b0187', 'b0188', 'b0189', 'b0190', 'b0191', 'b0192', 'b0193', 'b0194', 'b0195', 'b0196', 'b0197', 'b0198', 'b0199', 'b0200', 'b0201', 'b0202', 'b0203', 'b0204', 'b0205', 'b0206', 'b0207', 'b0208', 'b0209', 'b0210', 'b0211', 'b0212', 'b0213', 'b0214', 'b0215', 'b0216', 'b0217', 'b0218', 'b0219', 'b0220', 'b0221', 'b0222', 'b0223', 'b0224', 'b0225', 'b0226', 'b0227', 'b0228', 'b0229', 'b0230', 'b0231', 'b0232', 'b0233', 'b0234', 'b0235', 'b0236', 'b0237', 'b0238', 'b0239', 'b0240', 'b0241', 'b0242', 'b0243', 'b0244', 'b0245', 'b0246', 'b0247', 'b0248', 'b0249', 'b0250', 'b0251', 'b0252', 'b0253', 'b0254', 'b0256', 'b0258', 'b0259', 'b0260', 'b0261', 'b0262', 'b0263', 'b0264', 'b0265', 'b0266', 'b0267', 'b0268', 'b0269', 'b0270', 'b0271', 'b0272', 'b0273', 'b0274', 'b0275', 'b0276', 'b0277', 'b0278', 'b0279', 'b0280', 'b0281', 'b0283', 'b0284', 'b0285', 'b0286', 'b0287', 'b0288', 'b0289', 'b0290', 'b0291', 'b0292', 'b0293', 'b0294', 'b0295', 'b0296', 'b0297', 'b0298', 'b0299', 'b0300', 'b0301', 'b0303', 'b0304', 'b0305', 'b0306', 'b0307', 'b0308', 'b0310', 'b0311', 'b0312', 'b0313', 'b0314', 'b0315', 'b0316', 'b0317', 'b0318', 'b0319', 'b0320', 'b0321', 'b0323', 'b0324', 'b0325', 'b0326', 'b0327', 'b0328', 'b0329', 'b0330', 'b0331', 'b0333', 'b0334', 'b0335', 'b0336', 'b0337', 'b0338', 'b0339', 'b0340', 'b0341', 'b0342', 'b0343', 'b0344', 'b0345', 'b0346', 'b0347', 'b0348', 'b0349', 'b0350', 'b0351', 'b0352', 'b0353', 'b0354', 'b0355', 'b0356', 'b0357', 'b0358', 'b0360', 'b0361', 'b0363', 'b0364', 'b0365', 'b0366', 'b0367', 'b0368', 'b0369', 'b0372', 'b0373', 'b0375', 'b0376', 'b0377', 'b0378', 'b0379', 'b0380', 'b0381', 'b0382', 'b0383', 'b0384', 'b0385', 'b0386', 'b0387', 'b0388', 'b0389', 'b0390', 'b0391', 'b0392', 'b0393', 'b0394', 'b0396', 'b0397', 'b0398', 'b0399', 'b0400', 'b0401', 'b0402', 'b0403', 'b0404', 'b0405', 'b0406', 'b0407', 'b0408', 'b0409', 'b0410', 'b0411', 'b0412', 'b0413', 'b0414', 'b0415', 'b0416', 'b0417', 'b0418', 'b0419', 'b0420', 'b0421', 'b0422', 'b0423', 'b0424', 'b0425', 'b0426', 'b0427', 'b0428', 'b0429', 'b0430', 'b0431', 'b0432', 'b0433', 'b0434', 'b0435', 'b0436', 'b0437', 'b0438', 'b0439', 'b0440', 'b0441', 'b0442', 'b0443', 'b0444', 'b0445', 'b0446', 'b0447', 'b0448', 'b0449', 'b0450', 'b0451', 'b0452', 'b0453', 'b0454', 'b0455', 'b0456', 'b0457', 'b0458', 'b0459', 'b0460', 'b0461', 'b0462', 'b0463', 'b0464', 'b0465', 'b0466', 'b0467', 'b0468', 'b0469', 'b0470', 'b0471', 'b0472', 'b0473', 'b0474', 'b0475', 'b0476', 'b0477', 'b0478', 'b0479', 'b0480', 'b0481', 'b0482', 'b0483', 'b0484', 'b0485', 'b0486', 'b0487', 'b0488', 'b0489', 'b0490', 'b0491', 'b0492', 'b0493', 'b0494', 'b0495', 'b0496', 'b0497', 'b0498', 'b0499', 'b0501', 'b0502', 'b0503', 'b0504', 'b0505', 'b0506', 'b0507', 'b0508', 'b0509', 'b0511', 'b0512', 'b0513', 'b0514', 'b0515', 'b0516', 'b0517', 'b0518', 'b0520', 'b0521', 'b0522', 'b0523', 'b0524', 'b0525', 'b0526', 'b0527', 'b0528', 'b0529', 'b0530', 'b0531', 'b0532', 'b0533', 'b0534', 'b0535', 'b0536', 'b0537', 'b0539', 'b0540', 'b0541', 'b0542', 'b0543', 'b0544', 'b0545', 'b0546', 'b0547', 'b0548', 'b0549', 'b0550', 'b0551', 'b0552', 'b0553', 'b0554', 'b0555', 'b0556', 'b0557', 'b0558', 'b0559', 'b0560', 'b0561', 'b0562', 'b0563', 'b0564', 'b0565', 'b0566', 'b0567', 'b0568', 'b0569', 'b0570', 'b0571', 'b0572', 'b0573', 'b0574', 'b0575', 'b0576', 'b0577', 'b0578', 'b0579', 'b0580', 'b0581', 'b0582', 'b0583', 'b0584', 'b0585', 'b0586', 'b0587', 'b0588', 'b0589', 'b0590', 'b0591', 'b0592', 'b0593', 'b0594', 'b0595', 'b0596', 'b0597', 'b0598', 'b0599', 'b0600', 'b0601', 'b0602', 'b0603', 'b0604', 'b0605', 'b0606', 'b0607', 'b0608', 'b0610', 'b0611', 'b0612', 'b0613', 'b0614', 'b0615', 'b0616', 'b0617', 'b0618', 'b0619', 'b0620', 'b0621', 'b0622', 'b0623', 'b0624', 'b0627', 'b0628', 'b0629', 'b0630', 'b0631', 'b0632', 'b0633', 'b0634', 'b0635', 'b0636', 'b0637', 'b0638', 'b0639', 'b0640', 'b0641', 'b0642', 'b0643', 'b0644', 'b0645', 'b0646', 'b0647', 'b0648', 'b0649', 'b0650', 'b0651', 'b0652', 'b0653', 'b0654', 'b0655', 'b0656', 'b0657', 'b0658', 'b0659', 'b0660', 'b0661', 'b0662', 'b0664', 'b0665', 'b0666', 'b0668', 'b0670', 'b0672', 'b0673', 'b0674', 'b0675', 'b0676', 'b0677', 'b0678', 'b0679', 'b0680', 'b0681', 'b0682', 'b0683', 'b0684', 'b0685', 'b0686', 'b0687', 'b0688', 'b0689', 'b0691', 'b0692', 'b0693', 'b0694', 'b0695', 'b0696', 'b0697', 'b0698', 'b0699', 'b0700', 'b0702', 'b0703', 'b0704', 'b0705', 'b0706', 'b0707', 'b0708', 'b0709', 'b0710', 'b0711', 'b0712', 'b0713', 'b0714', 'b0715', 'b0716', 'b0717', 'b0718', 'b0719', 'b0720', 'b0721', 'b0722', 'b0723', 'b0724', 'b0726', 'b0727', 'b0728', 'b0729', 'b0730', 'b0731', 'b0732', 'b0733', 'b0734', 'b0735', 'b0736', 'b0737', 'b0738', 'b0739', 'b0740', 'b0741', 'b0742', 'b0743', 'b0744', 'b0745', 'b0746', 'b0747', 'b0748', 'b0749', 'b0750', 'b0751', 'b0752', 'b0753', 'b0754', 'b0755', 'b0756', 'b0757', 'b0758', 'b0759', 'b0760', 'b0761', 'b0762', 'b0763', 'b0764', 'b0765', 'b0766', 'b0767', 'b0768', 'b0769', 'b0770', 'b0771', 'b0772', 'b0773', 'b0774', 'b0775', 'b0776', 'b0777', 'b0778', 'b0779', 'b0780', 'b0781', 'b0782', 'b0783', 'b0784', 'b0785', 'b0786', 'b0787', 'b0788', 'b0789', 'b0790', 'b0791', 'b0792', 'b0793', 'b0794', 'b0795', 'b0796', 'b0797', 'b0798', 'b0799', 'b0800', 'b0801', 'b0802', 'b0803', 'b0804', 'b0805', 'b0806', 'b0807', 'b0808', 'b0809', 'b0810', 'b0811', 'b0812', 'b0813', 'b0814', 'b0815', 'b0817', 'b0818', 'b0819', 'b0820', 'b0821', 'b0822', 'b0823', 'b0824', 'b0825', 'b0826', 'b0827', 'b0828', 'b0829', 'b0830', 'b0831', 'b0832', 'b0833', 'b0834', 'b0835', 'b0836', 'b0837', 'b0838', 'b0839', 'b0840', 'b0841', 'b0842', 'b0843', 'b0844', 'b0845', 'b0846', 'b0847', 'b0848', 'b0849', 'b0850', 'b0851', 'b0852', 'b0853', 'b0854', 'b0855', 'b0856', 'b0857', 'b0858', 'b0859', 'b0860', 'b0861', 'b0862', 'b0863', 'b0864', 'b0865', 'b0866', 'b0867', 'b0868', 'b0869', 'b0870', 'b0871', 'b0872', 'b0873', 'b0874', 'b0875', 'b0876', 'b0877', 'b0878', 'b0879', 'b0880', 'b0881', 'b0882', 'b0883', 'b0884', 'b0885', 'b0886', 'b0887', 'b0888', 'b0889', 'b0890', 'b0891', 'b0892', 'b0893', 'b0894', 'b0895', 'b0896', 'b0897', 'b0898', 'b0899', 'b0900', 'b0901', 'b0902', 'b0903', 'b0904', 'b0905', 'b0906', 'b0907', 'b0908', 'b0909', 'b0910', 'b0911', 'b0912', 'b0913', 'b0914', 'b0915', 'b0916', 'b0917', 'b0918', 'b0919', 'b0920', 'b0921', 'b0922', 'b0923', 'b0924', 'b0925', 'b0926', 'b0927', 'b0928', 'b0929', 'b0930', 'b0931', 'b0932', 'b0933', 'b0934', 'b0935', 'b0936', 'b0937', 'b0938', 'b0939', 'b0940', 'b0941', 'b0942', 'b0943', 'b0944', 'b0945', 'b0946', 'b0947', 'b0948', 'b0949', 'b0950', 'b0951', 'b0952', 'b0953', 'b0954', 'b0955', 'b0956', 'b0957', 'b0958', 'b0959', 'b0960', 'b0961', 'b0962', 'b0963', 'b0964', 'b0965', 'b0966', 'b0967', 'b0968', 'b0969', 'b0970', 'b0971', 'b0972', 'b0973', 'b0974', 'b0975', 'b0976', 'b0977', 'b0978', 'b0979', 'b0980', 'b0981', 'b0982', 'b0983', 'b0984', 'b0985', 'b0986', 'b0987', 'b0988', 'b0989', 'b0990', 'b0991', 'b0992', 'b0993', 'b0994', 'b0995', 'b0996', 'b0997', 'b0998', 'b0999', 'b1000', 'b1001', 'b1002', 'b1003', 'b1004', 'b1006', 'b1007', 'b1008', 'b1009', 'b1010', 'b1011', 'b1012', 'b1013', 'b1014', 'b1015', 'b1018', 'b1019', 'b1020', 'b1021', 'b1022', 'b1023', 'b1024', 'b1025', 'b1026', 'b1027', 'b1028', 'b1029', 'b1032', 'b1033', 'b1034', 'b1035', 'b1036', 'b1037', 'b1038', 'b1039', 'b1040', 'b1041', 'b1042', 'b1043', 'b1044', 'b1045', 'b1046', 'b1047', 'b1048', 'b1049', 'b1050', 'b1051', 'b1053', 'b1054', 'b1055', 'b1056', 'b1057', 'b1058', 'b1059', 'b1060', 'b1061', 'b1062', 'b1063', 'b1064', 'b1065', 'b1066', 'b1067', 'b1068', 'b1069', 'b1070', 'b1071', 'b1072', 'b1073', 'b1074', 'b1075', 'b1076', 'b1077', 'b1078', 'b1079', 'b1080', 'b1081', 'b1082', 'b1083', 'b1084', 'b1085', 'b1086', 'b1087', 'b1088', 'b1089', 'b1090', 'b1091', 'b1092', 'b1093', 'b1094', 'b1095', 'b1096', 'b1097', 'b1098', 'b1099', 'b1100', 'b1101', 'b1102', 'b1103', 'b1104', 'b1105', 'b1106', 'b1107', 'b1108', 'b1109', 'b1110', 'b1111', 'b1112', 'b1113', 'b1114', 'b1115', 'b1116', 'b1117', 'b1118', 'b1119', 'b1120', 'b1121', 'b1122', 'b1123', 'b1124', 'b1125', 'b1126', 'b1127', 'b1128', 'b1129', 'b1130', 'b1131', 'b1132', 'b1133', 'b1134', 'b1135', 'b1136', 'b1137', 'b1138', 'b1139', 'b1140', 'b1141', 'b1143', 'b1144', 'b1145', 'b1146', 'b1147', 'b1148', 'b1150', 'b1151', 'b1152', 'b1153', 'b1154', 'b1155', 'b1156', 'b1157', 'b1158', 'b1159', 'b1160', 'b1161', 'b1162', 'b1163', 'b1164', 'b1165', 'b1166', 'b1167', 'b1168', 'b1171', 'b1172', 'b1174', 'b1175', 'b1176', 'b1177', 'b1178', 'b1179', 'b1180', 'b1181', 'b1182', 'b1183', 'b1184', 'b1185', 'b1186', 'b1187', 'b1188', 'b1189', 'b1190', 'b1191', 'b1192', 'b1193', 'b1194', 'b1195', 'b1196', 'b1197', 'b1198', 'b1199', 'b1200', 'b1201', 'b1202', 'b1203', 'b1204', 'b1205', 'b1206', 'b1207', 'b1208', 'b1209', 'b1210', 'b1211', 'b1212', 'b1213', 'b1214', 'b1215', 'b1216', 'b1217', 'b1218', 'b1219', 'b1220', 'b1221', 'b1222', 'b1223', 'b1224', 'b1225', 'b1226', 'b1227', 'b1229', 'b1230', 'b1231', 'b1232', 'b1233', 'b1234', 'b1235', 'b1236', 'b1237', 'b1238', 'b1241', 'b1242', 'b1243', 'b1244', 'b1245', 'b1246', 'b1247', 'b1248', 'b1249', 'b1250', 'b1251', 'b1252', 'b1253', 'b1254', 'b1255', 'b1256', 'b1257', 'b1258', 'b1259', 'b1260', 'b1261', 'b1262', 'b1263', 'b1264', 'b1265', 'b1266', 'b1267', 'b1268', 'b1269', 'b1270', 'b1271', 'b1272', 'b1273', 'b1274', 'b1275', 'b1276', 'b1277', 'b1278', 'b1279', 'b1280', 'b1281', 'b1282', 'b1283', 'b1284', 'b1285', 'b1286', 'b1287', 'b1288', 'b1289', 'b1290', 'b1291', 'b1292', 'b1293', 'b1294', 'b1295', 'b1296', 'b1297', 'b1298', 'b1299', 'b1300', 'b1301', 'b1302', 'b1303', 'b1304', 'b1305', 'b1306', 'b1307', 'b1308', 'b1309', 'b1310', 'b1311', 'b1312', 'b1313', 'b1314', 'b1315', 'b1316', 'b1317', 'b1319', 'b1320', 'b1321', 'b1322', 'b1323', 'b1324', 'b1325', 'b1326', 'b1327', 'b1328', 'b1329', 'b1330', 'b1331', 'b1332', 'b1333', 'b1334', 'b1335', 'b1336', 'b1337', 'b1338', 'b1339', 'b1340', 'b1341', 'b1342', 'b1343', 'b1344', 'b1345', 'b1346', 'b1347', 'b1348', 'b1349', 'b1350', 'b1351', 'b1352', 'b1353', 'b1355', 'b1356', 'b1357', 'b1358', 'b1359', 'b1360', 'b1361', 'b1362', 'b1363', 'b1365', 'b1366', 'b1368', 'b1370', 'b1372', 'b1373', 'b1374', 'b1375', 'b1376', 'b1377', 'b1378', 'b1379', 'b1380', 'b1381', 'b1382', 'b1383', 'b1384', 'b1385', 'b1386', 'b1387', 'b1388', 'b1389', 'b1390', 'b1391', 'b1392', 'b1393', 'b1394', 'b1395', 'b1396', 'b1397', 'b1398', 'b1399', 'b1400', 'b1402', 'b1403', 'b1404', 'b1406', 'b1407', 'b1408', 'b1409', 'b1410', 'b1411', 'b1412', 'b1413', 'b1414', 'b1415', 'b1418', 'b1419', 'b1420', 'b1421', 'b1422', 'b1423', 'b1424', 'b1426', 'b1427', 'b1428', 'b1429', 'b1430', 'b1431', 'b1432', 'b1433', 'b1434', 'b1435', 'b1436', 'b1438', 'b1439', 'b1440', 'b1441', 'b1442', 'b1443', 'b1444', 'b1445', 'b1446', 'b1447', 'b1448', 'b1449', 'b1450', 'b1451', 'b1452', 'b1453', 'b1454', 'b1455', 'b1456', 'b1457', 'b1459', 'b1460', 'b1461', 'b1462', 'b1463', 'b1464', 'b1465', 'b1466', 'b1467', 'b1468', 'b1469', 'b1471', 'b1472', 'b1473', 'b1474', 'b1475', 'b1476', 'b1477', 'b1478', 'b1479', 'b1480', 'b1481', 'b1482', 'b1483', 'b1484', 'b1485', 'b1486', 'b1487', 'b1488', 'b1489', 'b1490', 'b1491', 'b1492', 'b1493', 'b1494', 'b1495', 'b1496', 'b1497', 'b1498', 'b1499', 'b1500', 'b1501', 'b1502', 'b1503', 'b1504', 'b1505', 'b1506', 'b1507', 'b1508', 'b1511', 'b1512', 'b1513', 'b1514', 'b1515', 'b1516', 'b1517', 'b1518', 'b1519', 'b1520', 'b1521', 'b1522', 'b1523', 'b1524', 'b1525', 'b1526', 'b1527', 'b1528', 'b1529', 'b1530', 'b1531', 'b1532', 'b1533', 'b1534', 'b1535', 'b1536', 'b1537', 'b1538', 'b1539', 'b1540', 'b1541', 'b1542', 'b1544', 'b1545', 'b1546', 'b1547', 'b1548', 'b1549', 'b1550', 'b1551', 'b1552', 'b1553', 'b1554', 'b1555', 'b1556', 'b1557', 'b1558', 'b1559', 'b1560', 'b1561', 'b1562', 'b1563', 'b1564', 'b1565', 'b1566', 'b1567', 'b1568', 'b1569', 'b1570', 'b1571', 'b1572', 'b1573', 'b1574', 'b1575', 'b1576', 'b1577', 'b1578', 'b1579', 'b1580', 'b1581', 'b1582', 'b1583', 'b1584', 'b1585', 'b1586', 'b1587', 'b1588', 'b1589', 'b1590', 'b1591', 'b1592', 'b1593', 'b1594', 'b1595', 'b1596', 'b1597', 'b1598', 'b1599', 'b1600', 'b1601', 'b1602', 'b1603', 'b1604', 'b1605', 'b1606', 'b1607', 'b1608', 'b1609', 'b1610', 'b1611', 'b1612', 'b1613', 'b1614', 'b1615', 'b1616', 'b1617', 'b1618', 'b1619', 'b1620', 'b1621', 'b1622', 'b1623', 'b1624', 'b1625', 'b1626', 'b1627', 'b1628', 'b1629', 'b1630', 'b1631', 'b1632', 'b1633', 'b1634', 'b1635', 'b1636', 'b1637', 'b1638', 'b1639', 'b1640', 'b1641', 'b1642', 'b1643', 'b1644', 'b1645', 'b1646', 'b1647', 'b1648', 'b1649', 'b1650', 'b1651', 'b1652', 'b1653', 'b1654', 'b1655', 'b1656', 'b1657', 'b1658', 'b1659', 'b1660', 'b1661', 'b1662', 'b1663', 'b1664', 'b1665', 'b1666', 'b1667', 'b1668', 'b1669', 'b1670', 'b1671', 'b1672', 'b1673', 'b1674', 'b1675', 'b1676', 'b1677', 'b1678', 'b1679', 'b1680', 'b1681', 'b1682', 'b1683', 'b1684', 'b1685', 'b1686', 'b1687', 'b1688', 'b1689', 'b1690', 'b1691', 'b1692', 'b1693', 'b1694', 'b1695', 'b1696', 'b1697', 'b1698', 'b1699', 'b1700', 'b1701', 'b1702', 'b1703', 'b1704', 'b1705', 'b1706', 'b1707', 'b1708', 'b1709', 'b1710', 'b1711', 'b1712', 'b1713', 'b1714', 'b1715', 'b1716', 'b1717', 'b1718', 'b1719', 'b1722', 'b1723', 'b1724', 'b1725', 'b1726', 'b1727', 'b1728', 'b1729', 'b1730', 'b1731', 'b1732', 'b1733', 'b1734', 'b1735', 'b1736', 'b1737', 'b1738', 'b1739', 'b1740', 'b1741', 'b1742', 'b1743', 'b1744', 'b1745', 'b1746', 'b1747', 'b1748', 'b1749', 'b1750', 'b1751', 'b1752', 'b1753', 'b1754', 'b1755', 'b1756', 'b1757', 'b1758', 'b1759', 'b1760', 'b1761', 'b1762', 'b1763', 'b1764', 'b1765', 'b1766', 'b1767', 'b1768', 'b1769', 'b1770', 'b1771', 'b1772', 'b1773', 'b1774', 'b1775', 'b1776', 'b1777', 'b1778', 'b1779', 'b1780', 'b1781', 'b1782', 'b1783', 'b1784', 'b1785', 'b1786', 'b1787', 'b1788', 'b1789', 'b1790', 'b1791', 'b1792', 'b1793', 'b1794', 'b1795', 'b1796', 'b1797', 'b1798', 'b1799', 'b1800', 'b1801', 'b1802', 'b1803', 'b1804', 'b1805', 'b1806', 'b1807', 'b1808', 'b1809', 'b1810', 'b1811', 'b1812', 'b1813', 'b1814', 'b1815', 'b1816', 'b1817', 'b1818', 'b1819', 'b1820', 'b1821', 'b1822', 'b1823', 'b1824', 'b1825', 'b1826', 'b1827', 'b1828', 'b1829', 'b1830', 'b1831', 'b1832', 'b1833', 'b1834', 'b1835', 'b1836', 'b1837', 'b1838', 'b1839', 'b1840', 'b1841', 'b1842', 'b1843', 'b1844', 'b1845', 'b1846', 'b1847', 'b1848', 'b1849', 'b1850', 'b1851', 'b1852', 'b1853', 'b1854', 'b1855', 'b1856', 'b1857', 'b1858', 'b1859', 'b1860', 'b1861', 'b1862', 'b1863', 'b1864', 'b1865', 'b1866', 'b1867', 'b1868', 'b1869', 'b1870', 'b1871', 'b1872', 'b1873', 'b1874', 'b1875', 'b1876', 'b1877', 'b1878', 'b1879', 'b1880', 'b1881', 'b1882', 'b1883', 'b1884', 'b1885', 'b1886', 'b1887', 'b1888', 'b1889', 'b1890', 'b1891', 'b1892', 'b1893', 'b1894', 'b1895', 'b1896', 'b1897', 'b1900', 'b1901', 'b1902', 'b1904', 'b1905', 'b1906', 'b1907', 'b1908', 'b1909', 'b1910', 'b1911', 'b1912', 'b1913', 'b1914', 'b1915', 'b1916', 'b1917', 'b1918', 'b1919', 'b1920', 'b1921', 'b1922', 'b1923', 'b1924', 'b1925', 'b1926', 'b1927', 'b1928', 'b1929', 'b1930', 'b1931', 'b1932', 'b1936', 'b1937', 'b1938', 'b1939', 'b1940', 'b1941', 'b1942', 'b1943', 'b1944', 'b1945', 'b1946', 'b1947', 'b1948', 'b1949', 'b1950', 'b1951', 'b1952', 'b1953', 'b1954', 'b1955', 'b1956', 'b1957', 'b1958', 'b1959', 'b1960', 'b1961', 'b1962', 'b1963', 'b1967', 'b1968', 'b1969', 'b1970', 'b1971', 'b1972', 'b1973', 'b1974', 'b1975', 'b1976', 'b1977', 'b1978', 'b1981', 'b1982', 'b1983', 'b1984', 'b1985', 'b1986', 'b1987', 'b1988', 'b1989', 'b1990', 'b1991', 'b1992', 'b1993', 'b1994', 'b1996', 'b1997', 'b1999', 'b2000', 'b2001', 'b2002', 'b2003', 'b2004', 'b2005', 'b2006', 'b2007', 'b2008', 'b2009', 'b2010', 'b2011', 'b2012', 'b2013', 'b2014', 'b2015', 'b2016', 'b2017', 'b2018', 'b2019', 'b2020', 'b2021', 'b2022', 'b2023', 'b2024', 'b2025', 'b2026', 'b2027', 'b2028', 'b2029', 'b2030', 'b2032', 'b2033', 'b2034', 'b2035', 'b2036', 'b2037', 'b2038', 'b2039', 'b2040', 'b2041', 'b2042', 'b2043', 'b2044', 'b2045', 'b2046', 'b2047', 'b2048', 'b2049', 'b2050', 'b2051', 'b2052', 'b2053', 'b2054', 'b2055', 'b2056', 'b2057', 'b2058', 'b2059', 'b2060', 'b2061', 'b2062', 'b2063', 'b2064', 'b2065', 'b2066', 'b2067', 'b2068', 'b2069', 'b2070', 'b2071', 'b2072', 'b2073', 'b2074', 'b2075', 'b2076', 'b2077', 'b2078', 'b2079', 'b2080', 'b2081', 'b2082', 'b2083', 'b2085', 'b2086', 'b2088', 'b2089', 'b2091', 'b2092', 'b2093', 'b2094', 'b2095', 'b2096', 'b2097', 'b2098', 'b2099', 'b2100', 'b2101', 'b2102', 'b2103', 'b2104', 'b2105', 'b2106', 'b2107', 'b2108', 'b2109', 'b2110', 'b2111', 'b2112', 'b2113', 'b2114', 'b2118', 'b2119', 'b2120', 'b2121', 'b2122', 'b2123', 'b2124', 'b2125', 'b2126', 'b2127', 'b2128', 'b2129', 'b2130', 'b2131', 'b2132', 'b2133', 'b2134', 'b2135', 'b2136', 'b2137', 'b2139', 'b2140', 'b2141', 'b2142', 'b2143', 'b2144', 'b2145', 'b2146', 'b2147', 'b2148', 'b2149', 'b2150', 'b2151', 'b2152', 'b2153', 'b2154', 'b2155', 'b2156', 'b2157', 'b2158', 'b2159', 'b2160', 'b2161', 'b2162', 'b2163', 'b2164', 'b2165', 'b2166', 'b2167', 'b2168', 'b2169', 'b2170', 'b2171', 'b2172', 'b2173', 'b2174', 'b2175', 'b2176', 'b2177', 'b2178', 'b2179', 'b2180', 'b2181', 'b2182', 'b2183', 'b2184', 'b2185', 'b2186', 'b2187', 'b2188', 'b2189', 'b2190', 'b2192', 'b2193', 'b2194', 'b2195', 'b2196', 'b2197', 'b2198', 'b2199', 'b2200', 'b2201', 'b2202', 'b2203', 'b2204', 'b2205', 'b2206', 'b2207', 'b2208', 'b2209', 'b2210', 'b2211', 'b2212', 'b2213', 'b2214', 'b2215', 'b2216', 'b2217', 'b2218', 'b2219', 'b2220', 'b2221', 'b2222', 'b2223', 'b2224', 'b2225', 'b2226', 'b2229', 'b2230', 'b2231', 'b2232', 'b2233', 'b2234', 'b2235', 'b2236', 'b2237', 'b2238', 'b2239', 'b2240', 'b2241', 'b2242', 'b2243', 'b2244', 'b2245', 'b2246', 'b2247', 'b2248', 'b2249', 'b2250', 'b2251', 'b2252', 'b2253', 'b2254', 'b2255', 'b2256', 'b2257', 'b2258', 'b2259', 'b2260', 'b2261', 'b2262', 'b2263', 'b2264', 'b2265', 'b2266', 'b2267', 'b2268', 'b2269', 'b2270', 'b2271', 'b2272', 'b2273', 'b2274', 'b2275', 'b2276', 'b2277', 'b2278', 'b2279', 'b2280', 'b2281', 'b2282', 'b2283', 'b2284', 'b2285', 'b2286', 'b2287', 'b2288', 'b2289', 'b2290', 'b2291', 'b2292', 'b2293', 'b2294', 'b2295', 'b2296', 'b2297', 'b2298', 'b2299', 'b2300', 'b2301', 'b2302', 'b2303', 'b2304', 'b2305', 'b2306', 'b2307', 'b2308', 'b2309', 'b2310', 'b2311', 'b2312', 'b2313', 'b2314', 'b2315', 'b2316', 'b2317', 'b2318', 'b2319', 'b2320', 'b2321', 'b2322', 'b2323', 'b2324', 'b2325', 'b2326', 'b2327', 'b2328', 'b2329', 'b2330', 'b2331', 'b2332', 'b2333', 'b2334', 'b2335', 'b2336', 'b2339', 'b2340', 'b2341', 'b2342', 'b2343', 'b2344', 'b2345', 'b2346', 'b2347', 'b2348', 'b2349', 'b2350', 'b2351', 'b2352', 'b2353', 'b2354', 'b2355', 'b2356', 'b2357', 'b2358', 'b2359', 'b2360', 'b2361', 'b2362', 'b2363', 'b2364', 'b2365', 'b2366', 'b2367', 'b2368', 'b2369', 'b2370', 'b2371', 'b2372', 'b2373', 'b2374', 'b2375', 'b2376', 'b2377', 'b2378', 'b2379', 'b2380', 'b2381', 'b2382', 'b2383', 'b2384', 'b2385', 'b2386', 'b2387', 'b2388', 'b2389', 'b2390', 'b2392', 'b2393', 'b2394', 'b2395', 'b2396', 'b2397', 'b2398', 'b2399', 'b2400', 'b2401', 'b2402', 'b2403', 'b2404', 'b2405', 'b2406', 'b2407', 'b2408', 'b2409', 'b2410', 'b2411', 'b2412', 'b2413', 'b2414', 'b2415', 'b2416', 'b2417', 'b2418', 'b2419', 'b2420', 'b2421', 'b2422', 'b2423', 'b2424', 'b2425', 'b2426', 'b2427', 'b2428', 'b2429', 'b2430', 'b2431', 'b2432', 'b2433', 'b2434', 'b2435', 'b2436', 'b2437', 'b2438', 'b2439', 'b2440', 'b2441', 'b2442', 'b2443', 'b2444', 'b2445', 'b2446', 'b2447', 'b2448', 'b2449', 'b2450', 'b2451', 'b2452', 'b2453', 'b2454', 'b2455', 'b2456', 'b2457', 'b2458', 'b2459', 'b2460', 'b2461', 'b2462', 'b2463', 'b2464', 'b2465', 'b2466', 'b2467', 'b2468', 'b2469', 'b2470', 'b2471', 'b2472', 'b2473', 'b2474', 'b2475', 'b2476', 'b2477', 'b2478', 'b2479', 'b2480', 'b2481', 'b2482', 'b2483', 'b2484', 'b2485', 'b2486', 'b2487', 'b2488', 'b2489', 'b2490', 'b2491', 'b2492', 'b2493', 'b2494', 'b2495', 'b2496', 'b2497', 'b2498', 'b2499', 'b2500', 'b2501', 'b2502', 'b2503', 'b2504', 'b2505', 'b2506', 'b2507', 'b2508', 'b2509', 'b2510', 'b2511', 'b2512', 'b2513', 'b2514', 'b2515', 'b2516', 'b2517', 'b2518', 'b2519', 'b2520', 'b2521', 'b2522', 'b2523', 'b2524', 'b2525', 'b2526', 'b2527', 'b2528', 'b2529', 'b2530', 'b2531', 'b2532', 'b2533', 'b2534', 'b2535', 'b2536', 'b2537', 'b2538', 'b2539', 'b2540', 'b2541', 'b2542', 'b2543', 'b2544', 'b2545', 'b2546', 'b2547', 'b2548', 'b2549', 'b2550', 'b2551', 'b2552', 'b2553', 'b2554', 'b2555', 'b2556', 'b2557', 'b2558', 'b2559', 'b2560', 'b2561', 'b2562', 'b2563', 'b2564', 'b2565', 'b2566', 'b2567', 'b2568', 'b2569', 'b2570', 'b2571', 'b2572', 'b2573', 'b2574', 'b2575', 'b2576', 'b2577', 'b2578', 'b2579', 'b2580', 'b2581', 'b2582', 'b2583', 'b2584', 'b2585', 'b2586', 'b2587', 'b2588', 'b2589', 'b2590', 'b2591', 'b2592', 'b2593', 'b2594', 'b2595', 'b2597', 'b2598', 'b2599', 'b2600', 'b2601', 'b2602', 'b2603', 'b2604', 'b2605', 'b2606', 'b2607', 'b2608', 'b2609', 'b2610', 'b2611', 'b2614', 'b2615', 'b2616', 'b2617', 'b2618', 'b2619', 'b2620', 'b2621', 'b2622', 'b2623', 'b2624', 'b2625', 'b2626', 'b2627', 'b2628', 'b2629', 'b2630', 'b2631', 'b2632', 'b2633', 'b2634', 'b2635', 'b2636', 'b2637', 'b2638', 'b2641', 'b2642', 'b2643', 'b2644', 'b2645', 'b2646', 'b2647', 'b2648', 'b2650', 'b2652', 'b2659', 'b2660', 'b2661', 'b2662', 'b2663', 'b2664', 'b2665', 'b2666', 'b2667', 'b2668', 'b2669', 'b2670', 'b2671', 'b2672', 'b2673', 'b2674', 'b2675', 'b2676', 'b2677', 'b2678', 'b2679', 'b2681', 'b2682', 'b2683', 'b2684', 'b2685', 'b2686', 'b2687', 'b2688', 'b2689', 'b2690', 'b2691', 'b2692', 'b2693', 'b2694', 'b2695', 'b2696', 'b2697', 'b2698', 'b2699', 'b2700', 'b2701', 'b2702', 'b2703', 'b2704', 'b2705', 'b2706', 'b2707', 'b2708', 'b2709', 'b2710', 'b2711', 'b2712', 'b2713', 'b2714', 'b2715', 'b2716', 'b2717', 'b2718', 'b2719', 'b2720', 'b2721', 'b2722', 'b2723', 'b2724', 'b2725', 'b2726', 'b2727', 'b2728', 'b2729', 'b2730', 'b2731', 'b2732', 'b2733', 'b2734', 'b2735', 'b2736', 'b2737', 'b2738', 'b2739', 'b2740', 'b2741', 'b2742', 'b2743', 'b2744', 'b2745', 'b2746', 'b2747', 'b2748', 'b2749', 'b2750', 'b2751', 'b2752', 'b2753', 'b2754', 'b2755', 'b2756', 'b2757', 'b2758', 'b2759', 'b2760', 'b2761', 'b2762', 'b2763', 'b2764', 'b2765', 'b2766', 'b2767', 'b2768', 'b2769', 'b2770', 'b2771', 'b2774', 'b2775', 'b2776', 'b2777', 'b2778', 'b2779', 'b2780', 'b2781', 'b2782', 'b2783', 'b2784', 'b2785', 'b2786', 'b2787', 'b2788', 'b2789', 'b2790', 'b2791', 'b2792', 'b2793', 'b2794', 'b2795', 'b2796', 'b2797', 'b2798', 'b2799', 'b2800', 'b2801', 'b2802', 'b2803', 'b2804', 'b2805', 'b2806', 'b2807', 'b2808', 'b2809', 'b2810', 'b2811', 'b2812', 'b2813', 'b2814', 'b2815', 'b2816', 'b2817', 'b2818', 'b2819', 'b2820', 'b2821', 'b2822', 'b2823', 'b2824', 'b2825', 'b2826', 'b2827', 'b2828', 'b2829', 'b2830', 'b2831', 'b2832', 'b2833', 'b2834', 'b2835', 'b2836', 'b2837', 'b2838', 'b2839', 'b2840', 'b2841', 'b2842', 'b2843', 'b2844', 'b2845', 'b2846', 'b2847', 'b2848', 'b2849', 'b2850', 'b2851', 'b2852', 'b2853', 'b2854', 'b2856', 'b2858', 'b2859', 'b2860', 'b2861', 'b2863', 'b2864', 'b2865', 'b2866', 'b2867', 'b2868', 'b2869', 'b2870', 'b2871', 'b2872', 'b2873', 'b2874', 'b2875', 'b2876', 'b2877', 'b2878', 'b2879', 'b2880', 'b2881', 'b2882', 'b2883', 'b2886', 'b2887', 'b2888', 'b2889', 'b2890', 'b2891', 'b2892', 'b2893', 'b2894', 'b2895', 'b2896', 'b2897', 'b2898', 'b2899', 'b2900', 'b2901', 'b2902', 'b2903', 'b2904', 'b2905', 'b2906', 'b2907', 'b2908', 'b2909', 'b2910', 'b2911', 'b2912', 'b2913', 'b2914', 'b2915', 'b2916', 'b2917', 'b2918', 'b2919', 'b2920', 'b2921', 'b2922', 'b2923', 'b2924', 'b2925', 'b2926', 'b2927', 'b2928', 'b2929', 'b2930', 'b2933', 'b2934', 'b2935', 'b2936', 'b2937', 'b2938', 'b2939', 'b2940', 'b2942', 'b2943', 'b2944', 'b2945', 'b2946', 'b2947', 'b2948', 'b2949', 'b2950', 'b2951', 'b2952', 'b2953', 'b2954', 'b2955', 'b2956', 'b2957', 'b2958', 'b2959', 'b2960', 'b2961', 'b2962', 'b2963', 'b2964', 'b2965', 'b2966', 'b2967', 'b2968', 'b2969', 'b2970', 'b2971', 'b2972', 'b2975', 'b2976', 'b2977', 'b2979', 'b2980', 'b2981', 'b2982', 'b2983', 'b2984', 'b2985', 'b2986', 'b2987', 'b2988', 'b2989', 'b2990', 'b2991', 'b2992', 'b2993', 'b2994', 'b2995', 'b2996', 'b2997', 'b2998', 'b3001', 'b3002', 'b3003', 'b3005', 'b3006', 'b3008', 'b3009', 'b3010', 'b3011', 'b3012', 'b3013', 'b3014', 'b3017', 'b3018', 'b3019', 'b3020', 'b3021', 'b3022', 'b3023', 'b3024', 'b3025', 'b3026', 'b3027', 'b3028', 'b3029', 'b3030', 'b3031', 'b3032', 'b3033', 'b3034', 'b3035', 'b3037', 'b3038', 'b3039', 'b3040', 'b3041', 'b3042', 'b3043', 'b3044', 'b3045', 'b3046', 'b3047', 'b3048', 'b3049', 'b3050', 'b3051', 'b3052', 'b3053', 'b3054', 'b3055', 'b3056', 'b3057', 'b3058', 'b3059', 'b3060', 'b3061', 'b3062', 'b3063', 'b3064', 'b3065', 'b3066', 'b3067', 'b3068', 'b3069', 'b3070', 'b3071', 'b3072', 'b3073', 'b3074', 'b3075', 'b3076', 'b3077', 'b3078', 'b3079', 'b3080', 'b3081', 'b3082', 'b3083', 'b3084', 'b3085', 'b3086', 'b3087', 'b3088', 'b3089', 'b3090', 'b3091', 'b3092', 'b3093', 'b3094', 'b3095', 'b3096', 'b3097', 'b3098', 'b3099', 'b3100', 'b3101', 'b3102', 'b3103', 'b3104', 'b3105', 'b3106', 'b3107', 'b3110', 'b3113', 'b3114', 'b3115', 'b3116', 'b3117', 'b3118', 'b3119', 'b3120', 'b3121', 'b3123', 'b3124', 'b3125', 'b3126', 'b3127', 'b3128', 'b3129', 'b3130', 'b3131', 'b3132', 'b3133', 'b3134', 'b3135', 'b3136', 'b3137', 'b3138', 'b3139', 'b3140', 'b3141', 'b3142', 'b3143', 'b3144', 'b3145', 'b3146', 'b3147', 'b3148', 'b3149', 'b3150', 'b3151', 'b3152', 'b3153', 'b3154', 'b3155', 'b3156', 'b3157', 'b3158', 'b3159', 'b3160', 'b3161', 'b3162', 'b3163', 'b3164', 'b3165', 'b3166', 'b3167', 'b3168', 'b3169', 'b3170', 'b3171', 'b3172', 'b3173', 'b3174', 'b3175', 'b3176', 'b3177', 'b3178', 'b3179', 'b3180', 'b3181', 'b3182', 'b3183', 'b3184', 'b3185', 'b3186', 'b3187', 'b3188', 'b3189', 'b3190', 'b3191', 'b3192', 'b3193', 'b3194', 'b3195', 'b3196', 'b3197', 'b3198', 'b3199', 'b3200', 'b3201', 'b3202', 'b3203', 'b3204', 'b3205', 'b3206', 'b3207', 'b3208', 'b3209', 'b3210', 'b3211', 'b3212', 'b3213', 'b3214', 'b3215', 'b3216', 'b3218', 'b3219', 'b3220', 'b3221', 'b3222', 'b3223', 'b3224', 'b3225', 'b3226', 'b3227', 'b3228', 'b3229', 'b3230', 'b3231', 'b3232', 'b3233', 'b3234', 'b3235', 'b3236', 'b3237', 'b3238', 'b3239', 'b3240', 'b3241', 'b3242', 'b3243', 'b3244', 'b3247', 'b3248', 'b3249', 'b3250', 'b3251', 'b3252', 'b3253', 'b3255', 'b3256', 'b3257', 'b3258', 'b3259', 'b3260', 'b3261', 'b3262', 'b3263', 'b3264', 'b3265', 'b3266', 'b3267', 'b3268', 'b3269', 'b3270', 'b3271', 'b3272', 'b3273', 'b3274', 'b3275', 'b3276', 'b3277', 'b3278', 'b3279', 'b3280', 'b3281', 'b3282', 'b3283', 'b3284', 'b3287', 'b3288', 'b3289', 'b3290', 'b3291', 'b3292', 'b3293', 'b3294', 'b3295', 'b3296', 'b3297', 'b3298', 'b3299', 'b3300', 'b3301', 'b3302', 'b3303', 'b3304', 'b3305', 'b3306', 'b3307', 'b3308', 'b3309', 'b3310', 'b3311', 'b3312', 'b3313', 'b3314', 'b3315', 'b3316', 'b3317', 'b3318', 'b3319', 'b3320', 'b3321', 'b3322', 'b3323', 'b3324', 'b3325', 'b3326', 'b3327', 'b3328', 'b3329', 'b3330', 'b3331', 'b3332', 'b3333', 'b3334', 'b3335', 'b3336', 'b3337', 'b3338', 'b3339', 'b3340', 'b3341', 'b3342', 'b3343', 'b3344', 'b3345', 'b3346', 'b3347', 'b3348', 'b3349', 'b3350', 'b3351', 'b3352', 'b3353', 'b3354', 'b3355', 'b3356', 'b3357', 'b3358', 'b3359', 'b3360', 'b3361', 'b3362', 'b3363', 'b3364', 'b3365', 'b3366', 'b3367', 'b3368', 'b3369', 'b3370', 'b3371', 'b3374', 'b3375', 'b3376', 'b3377', 'b3378', 'b3379', 'b3380', 'b3381', 'b3382', 'b3383', 'b3384', 'b3385', 'b3386', 'b3387', 'b3388', 'b3389', 'b3390', 'b3391', 'b3392', 'b3393', 'b3394', 'b3395', 'b3396', 'b3397', 'b3398', 'b3399', 'b3400', 'b3401', 'b3402', 'b3403', 'b3404', 'b3405', 'b3406', 'b3407', 'b3408', 'b3409', 'b3410', 'b3411', 'b3412', 'b3413', 'b3414', 'b3415', 'b3416', 'b3417', 'b3418', 'b3421', 'b3422', 'b3423', 'b3424', 'b3425', 'b3426', 'b3427', 'b3428', 'b3429', 'b3430', 'b3431', 'b3432', 'b3433', 'b3434', 'b3437', 'b3438', 'b3439', 'b3440', 'b3441', 'b3442', 'b3443', 'b3444', 'b3445', 'b3446', 'b3447', 'b3448', 'b3449', 'b3450', 'b3451', 'b3452', 'b3453', 'b3454', 'b3455', 'b3456', 'b3457', 'b3458', 'b3459', 'b3460', 'b3461', 'b3462', 'b3463', 'b3464', 'b3465', 'b3466', 'b3467', 'b3468', 'b3469', 'b3470', 'b3471', 'b3472', 'b3473', 'b3474', 'b3475', 'b3476', 'b3477', 'b3478', 'b3479', 'b3480', 'b3481', 'b3482', 'b3483', 'b3484', 'b3485', 'b3486', 'b3487', 'b3488', 'b3491', 'b3492', 'b3493', 'b3494', 'b3495', 'b3496', 'b3497', 'b3498', 'b3499', 'b3500', 'b3501', 'b3502', 'b3503', 'b3504', 'b3505', 'b3506', 'b3507', 'b3508', 'b3509', 'b3510', 'b3511', 'b3512', 'b3513', 'b3514', 'b3515', 'b3516', 'b3517', 'b3518', 'b3519', 'b3520', 'b3521', 'b3522', 'b3523', 'b3524', 'b3525', 'b3526', 'b3527', 'b3528', 'b3529', 'b3530', 'b3531', 'b3532', 'b3533', 'b3534', 'b3535', 'b3536', 'b3537', 'b3538', 'b3539', 'b3540', 'b3541', 'b3542', 'b3543', 'b3544', 'b3545', 'b3546', 'b3547', 'b3548', 'b3549', 'b3550', 'b3551', 'b3552', 'b3553', 'b3554', 'b3555', 'b3556', 'b3557', 'b3558', 'b3559', 'b3560', 'b3561', 'b3562', 'b3563', 'b3564', 'b3565', 'b3566', 'b3567', 'b3568', 'b3569', 'b3570', 'b3571', 'b3572', 'b3573', 'b3574', 'b3575', 'b3576', 'b3577', 'b3578', 'b3579', 'b3580', 'b3581', 'b3582', 'b3583', 'b3584', 'b3585', 'b3586', 'b3587', 'b3588', 'b3589', 'b3590', 'b3591', 'b3592', 'b3593', 'b3594', 'b3595', 'b3596', 'b3597', 'b3598', 'b3599', 'b3600', 'b3601', 'b3602', 'b3603', 'b3604', 'b3605', 'b3606', 'b3607', 'b3608', 'b3609', 'b3610', 'b3611', 'b3612', 'b3613', 'b3614', 'b3615', 'b3616', 'b3617', 'b3618', 'b3619', 'b3620', 'b3621', 'b3622', 'b3623', 'b3624', 'b3625', 'b3626', 'b3627', 'b3628', 'b3629', 'b3630', 'b3631', 'b3632', 'b3633', 'b3634', 'b3635', 'b3636', 'b3637', 'b3638', 'b3639', 'b3640', 'b3641', 'b3642', 'b3643', 'b3644', 'b3645', 'b3646', 'b3647', 'b3648', 'b3649', 'b3650', 'b3651', 'b3652', 'b3653', 'b3654', 'b3655', 'b3656', 'b3657', 'b3658', 'b3659', 'b3660', 'b3661', 'b3662', 'b3663', 'b3664', 'b3665', 'b3666', 'b3667', 'b3668', 'b3669', 'b3670', 'b3671', 'b3672', 'b3673', 'b3674', 'b3675', 'b3676', 'b3677', 'b3678', 'b3679', 'b3680', 'b3683', 'b3684', 'b3685', 'b3686', 'b3687', 'b3688', 'b3689', 'b3690', 'b3691', 'b3693', 'b3696', 'b3697', 'b3698', 'b3699', 'b3700', 'b3701', 'b3702', 'b3703', 'b3704', 'b3705', 'b3706', 'b3707', 'b3708', 'b3709', 'b3710', 'b3711', 'b3712', 'b3713', 'b3714', 'b3715', 'b3716', 'b3717', 'b3718', 'b3719', 'b3720', 'b3721', 'b3722', 'b3723', 'b3724', 'b3725', 'b3726', 'b3727', 'b3728', 'b3729', 'b3730', 'b3731', 'b3732', 'b3733', 'b3734', 'b3735', 'b3736', 'b3737', 'b3738', 'b3739', 'b3740', 'b3741', 'b3742', 'b3743', 'b3744', 'b3745', 'b3746', 'b3747', 'b3748', 'b3749', 'b3750', 'b3751', 'b3752', 'b3753', 'b3754', 'b3755', 'b3756', 'b3757', 'b3758', 'b3759', 'b3760', 'b3761', 'b3764', 'b3765', 'b3766', 'b3769', 'b3770', 'b3771', 'b3772', 'b3773', 'b3774', 'b3775', 'b3777', 'b3778', 'b3779', 'b3780', 'b3781', 'b3783', 'b3784', 'b3785', 'b3786', 'b3787', 'b3788', 'b3789', 'b3790', 'b3791', 'b3792', 'b3793', 'b3794', 'b3795', 'b3796', 'b3797', 'b3798', 'b3799', 'b3800', 'b3801', 'b3802', 'b3803', 'b3804', 'b3805', 'b3806', 'b3807', 'b3809', 'b3810', 'b3811', 'b3812', 'b3813', 'b3816', 'b3817', 'b3818', 'b3819', 'b3820', 'b3821', 'b3822', 'b3823', 'b3824', 'b3825', 'b3826', 'b3827', 'b3828', 'b3829', 'b3830', 'b3831', 'b3832', 'b3833', 'b3834', 'b3835', 'b3836', 'b3838', 'b3839', 'b3842', 'b3843', 'b3844', 'b3845', 'b3846', 'b3847', 'b3848', 'b3849', 'b3850', 'b3851', 'b3852', 'b3853', 'b3854', 'b3855', 'b3856', 'b3857', 'b3858', 'b3859', 'b3860', 'b3861', 'b3862', 'b3863', 'b3864', 'b3865', 'b3866', 'b3867', 'b3868', 'b3869', 'b3870', 'b3871', 'b3872', 'b3873', 'b3874', 'b3875', 'b3876', 'b3877', 'b3878', 'b3879', 'b3880', 'b3881', 'b3882', 'b3883', 'b3884', 'b3885', 'b3886', 'b3887', 'b3888', 'b3889', 'b3890', 'b3891', 'b3892', 'b3893', 'b3894', 'b3895', 'b3896', 'b3897', 'b3898', 'b3899', 'b3900', 'b3901', 'b3902', 'b3903', 'b3904', 'b3905', 'b3906', 'b3907', 'b3908', 'b3909', 'b3910', 'b3911', 'b3912', 'b3915', 'b3916', 'b3917', 'b3918', 'b3919', 'b3920', 'b3921', 'b3922', 'b3923', 'b3924', 'b3925', 'b3926', 'b3927', 'b3928', 'b3929', 'b3930', 'b3931', 'b3932', 'b3933', 'b3934', 'b3935', 'b3936', 'b3937', 'b3938', 'b3939', 'b3940', 'b3941', 'b3942', 'b3943', 'b3944', 'b3945', 'b3946', 'b3947', 'b3949', 'b3950', 'b3951', 'b3952', 'b3953', 'b3954', 'b3955', 'b3956', 'b3957', 'b3958', 'b3959', 'b3960', 'b3961', 'b3962', 'b3963', 'b3964', 'b3965', 'b3966', 'b3967', 'b3968', 'b3969', 'b3970', 'b3971', 'b3972', 'b3973', 'b3974', 'b3976', 'b3977', 'b3978', 'b3979', 'b3980', 'b3981', 'b3982', 'b3983', 'b3984', 'b3985', 'b3986', 'b3987', 'b3988', 'b3989', 'b3990', 'b3991', 'b3992', 'b3993', 'b3994', 'b3995', 'b3996', 'b3997', 'b3998', 'b3999', 'b4000', 'b4001', 'b4002', 'b4003', 'b4004', 'b4005', 'b4006', 'b4007', 'b4008', 'b4009', 'b4010', 'b4011', 'b4012', 'b4013', 'b4014', 'b4015', 'b4016', 'b4017', 'b4018', 'b4019', 'b4020', 'b4021', 'b4022', 'b4023', 'b4024', 'b4025', 'b4026', 'b4027', 'b4028', 'b4029', 'b4030', 'b4031', 'b4032', 'b4033', 'b4034', 'b4035', 'b4036', 'b4037', 'b4038', 'b4039', 'b4040', 'b4041', 'b4042', 'b4043', 'b4044', 'b4045', 'b4046', 'b4047', 'b4048', 'b4049', 'b4050', 'b4051', 'b4052', 'b4053', 'b4054', 'b4055', 'b4056', 'b4057', 'b4058', 'b4059', 'b4060', 'b4061', 'b4062', 'b4063', 'b4064', 'b4065', 'b4066', 'b4067', 'b4068', 'b4069', 'b4070', 'b4071', 'b4072', 'b4073', 'b4074', 'b4075', 'b4076', 'b4077', 'b4078', 'b4079', 'b4080', 'b4081', 'b4082', 'b4083', 'b4084', 'b4085', 'b4086', 'b4087', 'b4088', 'b4089', 'b4090', 'b4092', 'b4093', 'b4094', 'b4095', 'b4096', 'b4097', 'b4098', 'b4099', 'b4100', 'b4101', 'b4102', 'b4104', 'b4105', 'b4106', 'b4107', 'b4108', 'b4109', 'b4110', 'b4111', 'b4112', 'b4113', 'b4114', 'b4115', 'b4116', 'b4117', 'b4118', 'b4119', 'b4120', 'b4121', 'b4122', 'b4123', 'b4124', 'b4125', 'b4126', 'b4127', 'b4128', 'b4129', 'b4130', 'b4131', 'b4132', 'b4133', 'b4134', 'b4135', 'b4136', 'b4137', 'b4138', 'b4139', 'b4140', 'b4141', 'b4142', 'b4143', 'b4144', 'b4145', 'b4146', 'b4147', 'b4148', 'b4149', 'b4150', 'b4151', 'b4152', 'b4153', 'b4154', 'b4155', 'b4156', 'b4157', 'b4158', 'b4159', 'b4160', 'b4161', 'b4162', 'b4163', 'b4164', 'b4165', 'b4166', 'b4167', 'b4168', 'b4169', 'b4170', 'b4171', 'b4172', 'b4173', 'b4174', 'b4175', 'b4176', 'b4177', 'b4178', 'b4179', 'b4180', 'b4181', 'b4182', 'b4183', 'b4184', 'b4185', 'b4186', 'b4187', 'b4188', 'b4189', 'b4190', 'b4191', 'b4192', 'b4193', 'b4194', 'b4195', 'b4196', 'b4197', 'b4198', 'b4199', 'b4200', 'b4201', 'b4202', 'b4203', 'b4204', 'b4205', 'b4206', 'b4207', 'b4208', 'b4209', 'b4210', 'b4211', 'b4212', 'b4213', 'b4214', 'b4215', 'b4216', 'b4217', 'b4218', 'b4219', 'b4220', 'b4221', 'b4222', 'b4224', 'b4225', 'b4226', 'b4227', 'b4230', 'b4231', 'b4232', 'b4233', 'b4234', 'b4235', 'b4236', 'b4237', 'b4238', 'b4239', 'b4240', 'b4241', 'b4242', 'b4243', 'b4244', 'b4245', 'b4246', 'b4248', 'b4249', 'b4251', 'b4252', 'b4253', 'b4254', 'b4255', 'b4256', 'b4257', 'b4258', 'b4259', 'b4260', 'b4261', 'b4262', 'b4263', 'b4264', 'b4265', 'b4266', 'b4267', 'b4268', 'b4269', 'b4270', 'b4271', 'b4272', 'b4273', 'b4277', 'b4278', 'b4279', 'b4280', 'b4281', 'b4282', 'b4284', 'b4286', 'b4287', 'b4288', 'b4289', 'b4290', 'b4291', 'b4292', 'b4293', 'b4294', 'b4295', 'b4296', 'b4297', 'b4298', 'b4299', 'b4300', 'b4301', 'b4302', 'b4303', 'b4304', 'b4305', 'b4306', 'b4307', 'b4308', 'b4309', 'b4310', 'b4311', 'b4312', 'b4313', 'b4314', 'b4315', 'b4316', 'b4317', 'b4318', 'b4319', 'b4320', 'b4321', 'b4322', 'b4323', 'b4324', 'b4325', 'b4326', 'b4327', 'b4328', 'b4329', 'b4330', 'b4331', 'b4332', 'b4333', 'b4334', 'b4335', 'b4336', 'b4337', 'b4339', 'b4340', 'b4341', 'b4342', 'b4345', 'b4346', 'b4347', 'b4348', 'b4349', 'b4350', 'b4351', 'b4352', 'b4353', 'b4354', 'b4355', 'b4356', 'b4357', 'b4358', 'b4359', 'b4360', 'b4361', 'b4362', 'b4363', 'b4364', 'b4365', 'b4366', 'b4367', 'b4368', 'b4369', 'b4370', 'b4371', 'b4372', 'b4373', 'b4374', 'b4375', 'b4376', 'b4377', 'b4378', 'b4379', 'b4380', 'b4381', 'b4382', 'b4383', 'b4384', 'b4385', 'b4386', 'b4387', 'b4388', 'b4389', 'b4390', 'b4391', 'b4392', 'b4393', 'b4394', 'b4395', 'b4396', 'b4397', 'b4398', 'b4399', 'b4400', 'b4401', 'b4402', 'b4403', 'b4406', 'b4407', 'b4408', 'b4409', 'b4410', 'b4411', 'b4412', 'b4413', 'b4414', 'b4415', 'b4417', 'b4418', 'b4419', 'b4420', 'b4421', 'b4422', 'b4423', 'b4424', 'b4425', 'b4426', 'b4427', 'b4428', 'b4429', 'b4430', 'b4431', 'b4432', 'b4433', 'b4435', 'b4436', 'b4437', 'b4438', 'b4439', 'b4440', 'b4441', 'b4442', 'b4443', 'b4444', 'b4445', 'b4446', 'b4447', 'b4449', 'b4450', 'b4451', 'b4452', 'b4453', 'b4454', 'b4455', 'b4456', 'b4457', 'b4458', 'b4459', 'b4460', 'b4461', 'b4462', 'b4463', 'b4464', 'b4465', 'b4466', 'b4467', 'b4468', 'b4469', 'b4470', 'b4471', 'b4472', 'b4473', 'b4474', 'b4475', 'b4476', 'b4477', 'b4478', 'b4479', 'b4480', 'b4481', 'b4482', 'b4483', 'b4484', 'b4485', 'b4486', 'b4487', 'b4488', 'b4490', 'b4491', 'b4492', 'b4493', 'b4494', 'b4495', 'b4496', 'b4497', 'b4498', 'b4499', 'b4500', 'b4501', 'b4502', 'b4503', 'b4504', 'b4505', 'b4506', 'b4508', 'b4509', 'b4510', 'b4511', 'b4512', 'b4513', 'b4514', 'b4515', 'b4516', 'b4517', 'b4518', 'b4519', 'b4520', 'b4521', 'b4522', 'b4523', 'b4524', 'b4525', 'b4526', 'b4527', 'b4528', 'b4529', 'b4532', 'b4533', 'b4535', 'b4536', 'b4537', 'b4538', 'b4539', 'b4541', 'b4542', 'b4543', 'b4544', 'b4545', 'b4546', 'b4547', 'b4548', 'b4550', 'b4551', 'b4552', 'b4553', 'b4554', 'b4555', 'b4556', 'b4557', 'b4558', 'b4559', 'b4561', 'b4565', 'b4566', 'b4567', 'b4568', 'b4569', 'b4570', 'b4571', 'b4572', 'b4573', 'b4575', 'b4576', 'b4577', 'b4578', 'b4579', 'b4580', 'b4581', 'b4582', 'b4585', 'b4586', 'b4587', 'b4588', 'b4589', 'b4590', 'b4592', 'b4593', 'b4594', 'b4595', 'b4596', 'b4597', 'b4598', 'b4599', 'b4600', 'b4601', 'b4602', 'b4603', 'b4606', 'b4608', 'b4609', 'b4611', 'b4613', 'b4614', 'b4615', 'b4616', 'b4618', 'b4620', 'b4621', 'b4622', 'b4623', 'b4624', 'b4625', 'b4626', 'b4627', 'b4628', 'b4629', 'b4630', 'b4631', 'b4632', 'b4633', 'b4634', 'b4635', 'b4636', 'b4637', 'b4638', 'b4639', 'b4640', 'b4641', 'b4642', 'b4643', 'b4644', 'b4645', 'b4646', 'b4647', 'b4648', 'b4649', 'b4650', 'b4651', 'b4652', 'b4653', 'b4654', 'b4655', 'b4656', 'b4657', 'b4658', 'b4659', 'b4660', 'b4661', 'b4662', 'b4663', 'b4664', 'b4665', 'b4666', 'b4667', 'b4668', 'b4669', 'b4670', 'b4671', 'b4672', 'b4673', 'b4674', 'b4675', 'b4676', 'b4677', 'b4678', 'b4679', 'b4680', 'b4682', 'b4683', 'b4684', 'b4685', 'b4686', 'b4687', 'b4688', 'b4689', 'b4690', 'b4691', 'b4692', 'b4693', 'b4694', 'b4695', 'b4696', 'b4697', 'b4698', 'b4699', 'b4700', 'b4701', 'b4702', 'b4703', 'b4704', 'b4705', 'b4706', 'b4707', 'b4708', 'b4712', 'b4713']

In [10]:
iM_dict = {}
iM_dict['resistance'] = ['translation_b1732', 'translation_b0871', 'translation_b2464', 'translation_b1897', 'translation_b2131', 'translation_b3073', 'translation_b1896', 'translation_b3447', 'translation_b0789', 'translation_b1478', 'translation_b2097', 'translation_b2130', 'translation_b1480', 'translation_b1927', 'translation_b2661', 'translation_b2128', 'translation_b2129', 'translation_b1197', 'translation_b2662', 'translation_b1519', 'translation_b3588', 'translation_b1440', 'translation_b1646', 'translation_b2741', 'translation_b2663', 'translation_b0837', 'translation_b0186', 'translation_b1441', 'translation_b3496', 'translation_b3012', 'translation_b2465', 'translation_b1519', 'translation_b1682', 'translation_b1681', 'translation_b1683', 'translation_b1680', 'translation_b1679', 'translation_b1684', 'translation_b3517', 'translation_b1493', 'translation_b1492', 'translation_b0485', 'translation_b3907']
iM_dict['persistence'] = ['translation_b1922', 'translation_b3924', 'translation_b1611', 'translation_b1378', 'translation_b1463', 'translation_b3908', 'translation_b1377', 'translation_b0871', 'translation_b3909', 'translation_b1852', 'translation_b2963', 'translation_b0684', 'translation_b1277', 'translation_b3519', 'translation_b2889', 'translation_b2895', 'translation_b1276', 'translation_b1533', 'translation_b0462', 'translation_b0628', 'translation_b2411', 'translation_b0168', 'translation_b0463', 'translation_b0578', 'translation_b0849', 'translation_b0931', 'translation_b0369', 'translation_b0096', 'translation_b0160', 'translation_b2964', 'translation_b3146', 'translation_b0870', 'translation_b2558', 'translation_b1851', 'translation_b0766', 'translation_b0124', 'translation_b3198', 'translation_b3917', 'translation_b3546', 'translation_b0767', 'translation_b3201', 'translation_b3035', 'translation_b2174', 'translation_b0469', 'translation_b3200', 'translation_b3199', 'translation_b1101', 'translation_b1215', 'translation_b0929', 'translation_b3666', 'translation_b0763', 'translation_b0765', 'translation_b0764', 'translation_b1922', 'translation_b0014', 'translation_b4143', 'translation_b4142', 'translation_b0015', 'translation_b2614', 'translation_b3343', 'translation_b1779', 'translation_b1380', 'translation_b1114', 'translation_b4171', 'translation_b3179', 'translation_b4034', 'translation_b3458', 'translation_b3457', 'translation_b3460', 'translation_b3456', 'translation_b3455', 'translation_b3454', 'translation_b0077', 'translation_b0078', 'translation_b3774', 'translation_b3829', 'translation_b1602', 'translation_b1603', 'translation_b2290', 'translation_b0072', 'translation_b0929', 'translation_b0863', 'translation_b0861', 'translation_b1907', 'translation_b0381', 'translation_b1814', 'translation_b4129', 'translation_b3496', 'translation_b1605', 'translation_b3616', 'translation_b2797', 'translation_b3617', 'translation_b2796', 'translation_b1634', 'translation_b1189', 'translation_b1190', 'translation_b2143', 'translation_b3831', 'translation_b4381', 'translation_b4382', 'translation_b2964', 'translation_b0411', 'translation_b3366', 'translation_b3365']

In [11]:
#simulate with proteome fraction constraints
create_reaction_sum_of_expression(me,'pathway_rproteins',rproteins)
create_reaction_sum_of_expression(me,'pathway_TCA',TCA)
create_reaction_sum_of_expression(me,'pathway_glycolysis',glycolysis)
create_reaction_sum_of_expression(me,'pathway_PPP',PPP)

b3851 Not modeled
b3855 Not modeled
b3854 Not modeled
b1675 Not modeled
b2929 Not modeled


In [12]:
#knock out BW25113 genes

KO_genes = ['b3126', 'b3127', 'b3128', 'b0344', 'b1241', 'b1251', 'b1252']

for i in KO_genes:
    try:
        me.reactions.get_by_id('translation_' + str(i)).knock_out()
    except:
        pass
print('genes knocked out')

genes knocked out


In [13]:
df_model = pd.DataFrame(index=['WT','HMP3427'],columns=['rprotein_mf','glycolysis_mf','PPP_mf','TCA_mf','growth_rate','glucose_uptake','oxygen_uptake','melatonin_secretion','acetate_secretion','formate_secretion','lactate_secretion','pyruvate_secretion','ethanol_secretion'])

df_model.loc[:,'growth_rate'] = 0.1

# All data averaged from 5-20h Fed-Batch
# glucose_uptake_rate [mmol/gDW*hr] = D-glucose consumed [mol/cmol(X)*h] * Conversion [mmol/mol] / Cell Molecular Mass [gDW/cmol(X)]
df_model.loc['WT','glucose_uptake'] = -0.029 * 1000 / 24.58
df_model.loc['HMP3427','glucose_uptake'] = -0.045 * 1000 / 24.58

# oxygen_uptake_rate [mmol/gDW*hr] =  [mol/cmol(X)*h] * Conversion [mmol/mol] / Cell Molecular Mass [gDW/cmol(X)]
df_model.loc['WT','co2_formation'] = 0.06 * 1000 / 24.58
df_model.loc['HMP3427','co2_formation'] = 0.15 * 1000 / 24.58

# melatonin_secretion_rate [mmol/gDW*hr] = melatonin specific formation rate molar [mol/cmol(X)*h] * Conversion [mmol/mol] / Cell Molecular Mass [gDW/cmol(X)]
df_model.loc['WT','melatonin_secretion'] = 0
df_model.loc['HMP3427','melatonin_secretion'] = -0.0005 * 1000 / 24.58

df_model.loc[:,'acetate_secretion'] = 0
df_model.loc[:,'formate_secretion'] = 0
df_model.loc[:,'lactate_secretion'] = 0
df_model.loc[:,'pyruvate_secretion'] = 0
df_model.loc[:,'ethanol_secretion'] = 0

df_model.head(2)

,rprotein_mf,glycolysis_mf,PPP_mf,TCA_mf,growth_rate,glucose_uptake,oxygen_uptake,melatonin_secretion,acetate_secretion,formate_secretion,lactate_secretion,pyruvate_secretion,ethanol_secretion,co2_formation
WT,NaN,NaN,NaN,NaN,0.1,-1.17982,NaN,0,0,0,0,0,0,2.441009
HMP3427,NaN,NaN,NaN,NaN,0.1,-1.83076,NaN,-0.0203417,0,0,0,0,0,6.102522


In [14]:
# Set Strain
_condition_list = df_model.index
_condition = _condition_list[0]

# # growth media conditions                                                                                                  CHECK

MU_PREC = 1e-4
MU_MIN  = df_model.loc[_condition,'growth_rate']
MU_MAX  = df_model.loc[_condition,'growth_rate']

# Flux conditions

rxn = me.reactions.EX_co2_e
rxn.lower_bound = -1000
rxn.upper_bound = 1000

rxn = me.reactions.EX_for_e
rxn.lower_bound = -1000
rxn.upper_bound = 1000

rxn = me.reactions.EX_ac_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_for_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_lac__D_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_pyr_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_etoh_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_acald_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_succ_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_akg_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_cit_e
rxn.lower_bound = 0
rxn.upper_bound = 0

rxn = me.reactions.EX_akg_e
rxn.lower_bound = 0
rxn.upper_bound = 0

# set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
# set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
# set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])

In [ ]:
# #basal first model for basis
ros_dict = {'h2o2_c': 5e-08, 'o2s_c': 2e-10, 'h2o2_e': 1.5e-06,
            'o2s_e': 0, 'pq2_c': 0}
stress.substitute_ros(solver, ros_dict)
stress.substitute_metal(solver)
# stress.couple_iron_conc()
sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX) #constrain with Mu
# sol = solver.bisectmu(precision=MU_PREC) #constrain without Mu
#set first basis for faster solving
hs = sol[1]
_damage = get_sum_damage(me.solution)
print(f'Damage: {_damage*100}%')

In [ ]:
# _list_scaling = [1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10,12,14,16,18,20,22,25]
_list_scaling = [0,5,10,20,25,30,35,40]
_base_condition = "WT"

df_ros = pd.DataFrame(columns=['mu','scaling','model_glucose','model_co2','model_o2','proteome_damage','resistance','persistence'])

for _scaling in _list_scaling:
    _condition = _base_condition
    _idx = str(_base_condition)+'_'+str(_scaling)

    # ros_dict = {'h2o2_c': 5e-08*_scaling, 'o2s_c': 2e-10*_scaling, 'h2o2_e': 1.5e-08*_scaling,
    #                 'o2s_e': 0, 'pq2_c': 0}
    ros_dict = {'h2o2_c': 5e-08, 'o2s_c': 2e-10, 'h2o2_e': 1.5e-06,
            'o2s_e': 0, 'pq2_c': 0}

    MU_PREC = 1e-4
    MU_MIN  = df_model.loc[_condition,'growth_rate']
    MU_MAX  = df_model.loc[_condition,'growth_rate']

    rxn = me.reactions.ATPM
    rxn.lower_bound = _scaling
    rxn.upper_bound = 1000

    rxn = me.reactions.EX_glc__D_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 0

    rxn = me.reactions.EX_for_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000

    rxn = me.reactions.EX_co2_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000

    rxn = me.reactions.EX_ac_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_for_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_lac__D_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_pyr_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_etoh_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_acald_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_succ_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_akg_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_cit_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_akg_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
    set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
    set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])

    stress.substitute_ros(solver, ros_dict)
    stress.substitute_metal(solver)

    sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX, basis = hs) #force growth rate

    hs = sol[1]

    _damage = get_sum_damage(me.solution)
    _resistance = get_sum(me.solution,iM_dict['resistance'])
    _persistence = get_sum(me.solution,iM_dict['persistence'])

    df_ros.loc[_idx,'mu'] = df_model.loc[_condition,'growth_rate']
    df_ros.loc[_idx,'scaling'] = _scaling
    df_ros.loc[_idx,'model_glucose'] = me.solution.x_dict['EX_glc__D_e']
    df_ros.loc[_idx,'model_co2'] = me.solution.x_dict['EX_co2_e']
    df_ros.loc[_idx,'model_o2'] = me.solution.x_dict['EX_o2_e']
    df_ros.loc[_idx,'proteome_damage'] = _damage
    df_ros.loc[_idx,'resistance'] = _resistance
    df_ros.loc[_idx,'persistence'] = _persistence

    filename = 'html_files/ros_'+str(_condition)+'_mu_'+str(df_model.loc[_condition,'growth_rate'])+'_scaling_'+str(_scaling)
#     solutionfile=str(filename)+'.pickle'
#     with open(solutionfile,'wb') as f:
#         pickle.dump(me,f)
    htmlfile = str(filename) + '.html'
    show_escher_map(stress).save_html(htmlfile)

df_ros #.head()

iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.541000 seconds
Finished makeME_LP in 1.020024 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.36943 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.535917 seconds
Finished makeME_LP in 1.014492 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.6728 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.517251 seconds
Finished makeME_LP in 1.016908 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.36631 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.553967 seconds
Finished makeME_LP in 1.017534 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.45809 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substi

,mu,scaling,model_glucose,model_co2,model_o2,proteome_damage,resistance,persistence
WT_0,0.1,0,-1.31012,3.42973,-3.10425,0.00694509,0.000812417,0.007791
WT_5,0.1,5,-1.66924,5.57432,-5.2516,0.00822515,0.00124157,0.0115012
WT_10,0.1,10,-2.02836,7.7189,-7.39893,0.00950526,0.00181225,0.015212
WT_20,0.1,20,-2.7466,12.0081,-11.6936,0.0120662,0.0029539,0.0226355
WT_25,0.1,25,-3.10572,14.1526,-13.841,0.013347,0.00352488,0.0263483
WT_30,0.1,30,-3.46484,16.2972,-15.9883,0.014628,0.00409596,0.0300618
WT_35,0.1,35,-3.82397,18.4418,-18.1356,0.0159093,0.00466714,0.0337759
WT_40,0.1,40,-4.18309,20.5864,-20.283,0.0171907,0.00523842,0.0374906


In [17]:
# _list_scaling = [1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10,12,14,16,18,20,22,25]
_list_scaling = [0,5,10,20,25,30,35,40]
_base_condition = "WT"

df_ros = pd.DataFrame(columns=['mu','scaling','model_glucose','model_co2','model_o2','proteome_damage','resistance','persistence'])

for _scaling in _list_scaling:
    _condition = _base_condition
    _idx = str(_base_condition)+'_'+str(_scaling)

    _ros_scaling = (_scaling*0.239)+1
    ros_dict = {'h2o2_c': 5e-08*_ros_scaling, 'o2s_c': 2e-10*_ros_scaling, 'h2o2_e': 1.5e-08*_ros_scaling,
                    'o2s_e': 0, 'pq2_c': 0}
    # ros_dict = {'h2o2_c': 5e-08, 'o2s_c': 2e-10, 'h2o2_e': 1.5e-06,
    #         'o2s_e': 0, 'pq2_c': 0}

    MU_PREC = 1e-4
    MU_MIN  = df_model.loc[_condition,'growth_rate']
    MU_MAX  = df_model.loc[_condition,'growth_rate']

    rxn = me.reactions.ATPM
    rxn.lower_bound = _scaling
    rxn.upper_bound = 1000

    rxn = me.reactions.EX_glc__D_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 0

    rxn = me.reactions.EX_for_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000

    rxn = me.reactions.EX_co2_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000

    rxn = me.reactions.EX_ac_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_for_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_lac__D_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_pyr_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_etoh_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_acald_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_succ_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_akg_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_cit_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_akg_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    # set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
    # set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
    # set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])

    stress.substitute_ros(solver, ros_dict)
    stress.substitute_metal(solver)

    sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX, basis = hs) #force growth rate

    hs = sol[1]

    _damage = get_sum_damage(me.solution)
    _resistance = get_sum(me.solution,iM_dict['resistance'])
    _persistence = get_sum(me.solution,iM_dict['persistence'])

    df_ros.loc[_idx,'mu'] = df_model.loc[_condition,'growth_rate']
    df_ros.loc[_idx,'scaling'] = _scaling
    df_ros.loc[_idx,'model_glucose'] = me.solution.x_dict['EX_glc__D_e']
    df_ros.loc[_idx,'model_co2'] = me.solution.x_dict['EX_co2_e']
    df_ros.loc[_idx,'model_o2'] = me.solution.x_dict['EX_o2_e']
    df_ros.loc[_idx,'proteome_damage'] = _damage
    df_ros.loc[_idx,'resistance'] = _resistance
    df_ros.loc[_idx,'persistence'] = _persistence

    filename = 'html_files/rosscale_'+str(_condition)+'_mu_'+str(df_model.loc[_condition,'growth_rate'])+'_scaling_'+str(_scaling)
#     solutionfile=str(filename)+'.pickle'
#     with open(solutionfile,'wb') as f:
#         pickle.dump(me,f)
    htmlfile = str(filename) + '.html'
    show_escher_map(stress).save_html(htmlfile)

df_ros #.head()

iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.506470 seconds
Finished makeME_LP in 1.017936 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 13.6582 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.482540 seconds
Finished makeME_LP in 1.010668 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 10.3255 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.506206 seconds
Finished makeME_LP in 1.017587 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.39435 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.512932 seconds
Finished makeME_LP in 1.024558 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.40538 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished subst

,mu,scaling,model_glucose,model_co2,model_o2,proteome_damage,resistance,persistence
WT_0,0.1,0,-1.31012,3.42973,-3.10425,0.00694509,0.000812417,0.007791
WT_5,0.1,5,-1.82516,6.50141,-6.18145,0.0112991,0.00306359,0.0122394
WT_10,0.1,10,-2.23809,8.95893,-8.64468,0.0174403,0.00372486,0.015798
WT_20,0.1,20,-3.06396,13.8739,-13.5711,0.0297013,0.00504798,0.0229189
WT_25,0.1,25,-3.47688,16.3314,-16.0343,0.035821,0.00570983,0.026481
WT_30,0.1,30,-3.88981,18.7889,-18.4975,0.0419336,0.00637188,0.0300444
WT_35,0.1,35,-4.30273,21.2464,-20.9607,0.0480385,0.00703531,0.0336089
WT_40,0.1,40,-4.71565,23.7039,-23.4239,0.0541365,0.0076979,0.0371746


In [29]:
solver.substitution_dict['h2o2_c']

{'mRNA_mean_lifetime_minutes': 5.0,
 'lol_efficiency_in_per_second': 0.9,
 'bam_efficiency_in_per_second': 0.0267,
 'tat_translocation_efficiency_per_second': 0.0125,
 'secA_translocation_efficiency_aa_per_second': 4.0,
 'proportion_of_rna_that_is_mrna': 0.02,
 'proportion_of_rna_that_is_trna': 0.12,
 'proportion_of_rna_that_is_rrna': 0.86,
 'average_rna_nt_mw': 324.0,
 'average_aa_mw': 109.0,
 'mass_rrna_per_ribosome': 1700000.0,
 'average_trna_mw': 25000.0,
 'enzyme_efficiency_scaling': 1.0,
 'percent_nadh_dehydrogenase_that_ndh1': 0.5,
 'unmodeled_protein_proportion_of_proteome': 0.36,
 'gam_value': 34.98,
 'proportion_of_inner_membrane_sa_that_is_protein': 0.75,
 'proportion_of_outer_membrane_sa_that_is_protein': 0.75,
 'ngam_value': 1.0,
 'mu': 0.1,
 h2o2_c: 3.726e-07,
 o2s_c: 1.4904000000000001e-09,
 h2o2_e: 1.1177999999999999e-07,
 o2s_e: 0,
 pq2_c: 0,
 fe2_c: 1e-05,
 zn2_c: 0.0001,
 mn2_c: 1e-05,
 B_mn2_c: 10000000000.0,
 B_fe2_c: 100000000000.0,
 B_zn2_c: 1000000000000.0}

In [ ]:
# _list_scaling = [1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10,12,14,16,18,20,22,25]
_list_scaling = [0,5,10,20,25,30,35,40]
_base_condition = "WT"
_base_oxygen_uptake = df_model.loc[_base_condition,'oxygen_uptake']
_base_glucose_uptake = df_model.loc[_base_condition,'glucose_uptake']

df_ros = pd.DataFrame(columns=['scaling','oxygen_uptake','glucose_uptake','model_co2','proteome_damage','resistance','persistence'])

for _scaling in _list_scaling:
    _condition = _base_condition
    _idx = str(_base_condition)+'_'+str(_scaling)
    # _oxygen_uptake = _scaling * _base_oxygen_uptake
    # _do2_uptake = ((_o2 * _base_o2_uptake) - _base_o2_uptake)
    # _glucose_uptake = _base_glucose_uptake + (((_scaling * _base_oxygen_uptake) - _base_oxygen_uptake)/6)
    # _glucose_uptake = _base_glucose_uptake
    ros_dict = {'h2o2_c': 5e-08*_scaling, 'o2s_c': 2e-10*_scaling, 'h2o2_e': 1.5e-08*_scaling,
                    'o2s_e': 0, 'pq2_c': 0}
    
    MU_PREC = 1e-4
    MU_MIN  = df_model.loc[_condition,'growth_rate']
    MU_MAX  = df_model.loc[_condition,'growth_rate']
    
    rxn = me.reactions.EX_glc__D_e
    rxn.lower_bound = _glucose_uptake-0.10
    rxn.upper_bound = _glucose_uptake+0.10
    
    rxn = me.reactions.EX_for_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000

    # rxn = me.reactions.EX_co2_e
    # rxn.lower_bound = _scaling * (-df_model.loc[_condition,'oxygen_uptake'])-0.2
    # rxn.upper_bound = _scaling * (-df_model.loc[_condition,'oxygen_uptake'])+0.2
    
    rxn = me.reactions.EX_co2_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    
    rxn = me.reactions.ATPM
    rxn.lower_bound = _scaling
    rxn.upper_bound = 1000
    
    # set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
    # set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
    # set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])
    
    stress.substitute_ros(solver, ros_dict)
    stress.substitute_metal(solver)

    sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX, basis = hs) #force growth rate

    hs = sol[1]
    
    _damage = get_sum_damage(me.solution)
    _resistance = get_sum(me.solution,iM_dict['resistance'])
    _persistence = get_sum(me.solution,iM_dict['persistence'])
    
    # print(f'{_condition} Damage: {get_sum_damage(_damage)}')
    
    df_ros.loc[_idx,'scaling'] = _scaling
    df_ros.loc[_idx,'oxygen_uptake'] = _oxygen_uptake
    df_ros.loc[_idx,'glucose_uptake'] = _glucose_uptake
    df_ros.loc[_idx,'model_co2'] = _glucose_uptake
    df_ros.loc[_idx,'proteome_damage'] = _damage
    df_ros.loc[_idx,'resistance'] = _resistance
    df_ros.loc[_idx,'persistence'] = _persistence
    
    filename = 'html_files/ros_'+str(_condition)+'_scaling_'+str(_scaling)+'maintenance'
#     solutionfile=str(filename)+'.pickle'                                                        
#     with open(solutionfile,'wb') as f:
#         pickle.dump(me,f)
    htmlfile = str(filename) + '.html'
    show_escher_map(stress).save_html(htmlfile)

df_ros.head()

iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.341481 seconds
Finished makeME_LP in 1.026911 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 46.7243 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.374234 seconds
Finished makeME_LP in 1.033614 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 18.8568 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.434093 seconds
Finished makeME_LP in 1.032799 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 12.5249 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.324428 seconds
Finished makeME_LP in 1.022771 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 15.3942 seconds


,scaling,oxygen_uptake,glucose_uptake,proteome_damage,resistance,persistence
WT_1,1,-2.44101,-1.17982,0.00515149,0.00054553,0.00842116
WT_5,5,-12.205,-2.80716,0.0257255,0.000352619,0.0127209
WT_10,10,-24.4101,-4.84133,0.0513594,0.000353996,0.0189605
WT_20,20,-48.8202,-8.90968,0.102346,0.000370257,0.0320623


In [111]:
# _list_scaling = [1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10,12,14,16,18,20,22,25]
_list_scaling = [1,5,10,15]
_base_condition = "HMP3427"
_base_oxygen_uptake = df_model.loc[_base_condition,'oxygen_uptake']
_base_glucose_uptake = df_model.loc[_base_condition,'glucose_uptake']

df_ros = pd.DataFrame(columns=['scaling','oxygen_uptake','glucose_uptake','model_o2','model_co2','proteome_damage','resistance','persistence'])

for _scaling in _list_scaling:
    _condition = _base_condition
    _idx = str(_base_condition)+'_'+str(_scaling)
    # _oxygen_uptake = _scaling * _base_oxygen_uptake
    # _do2_uptake = ((_o2 * _base_o2_uptake) - _base_o2_uptake)
    # _glucose_uptake = _base_glucose_uptake + (((_scaling * _base_oxygen_uptake) - _base_oxygen_uptake)/6)
    _glucose_uptake = _base_glucose_uptake

    MU_PREC = 1e-4
    MU_MIN  = df_model.loc[_condition,'growth_rate']
    MU_MAX  = df_model.loc[_condition,'growth_rate']

    rxn = me.reactions.EX_glc__D_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 0 # 1000

    rxn = me.reactions.ATPM
    rxn.lower_bound = _scaling
    rxn.upper_bound = 1000

    ros_dict = {'h2o2_c': 5e-08, 'o2s_c': 2e-10, 'h2o2_e': 1.5e-08,
                    'o2s_e': 0, 'pq2_c': 0}
    # ros_dict = {'h2o2_c': 5e-08*_scaling, 'o2s_c': 2e-10*_scaling, 'h2o2_e': 1.5e-08*_scaling,
    #                 'o2s_e': 0, 'pq2_c': 0}

    rxn = me.reactions.EX_for_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000

    # rxn = me.reactions.EX_co2_e
    # rxn.lower_bound = (_scaling * (-df_model.loc[_condition,'oxygen_uptake']))-0.1
    # rxn.upper_bound = (_scaling * (-df_model.loc[_condition,'oxygen_uptake']))+0.1
    # if _scaling > 1:
    #     rxn.lower_bound = 0
    #     rxn.upper_bound = 1000
    
    rxn = me.reactions.EX_co2_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    
    rxn = me.reactions.EX_ac_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_for_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_lac__D_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_pyr_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_etoh_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_acald_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_succ_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_akg_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_cit_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0

    rxn = me.reactions.EX_akg_e
    rxn.lower_bound = 0
    rxn.upper_bound = 0
    
    # set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
    # set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
    # set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])
    
    stress.substitute_ros(solver, ros_dict)
    stress.substitute_metal(solver)

    sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX, basis = hs) #force growth rate

    hs = sol[1]
    
    _damage = get_sum_damage(me.solution)
    _resistance = get_sum(me.solution,iM_dict['resistance'])
    _persistence = get_sum(me.solution,iM_dict['persistence'])
    
    # print(f'{_condition} Damage: {get_sum_damage(_damage)}')
    
    df_ros.loc[_idx,'scaling'] = _scaling
    df_ros.loc[_idx,'oxygen_uptake'] = _oxygen_uptake
    # df_ros.loc[_idx,'glucose_uptake'] = _glucose_uptake
    df_ros.loc[_idx,'glucose_uptake'] = me.solution.x_dict['EX_glc__D_e']
    df_ros.loc[_idx,'model_co2'] = me.solution.x_dict['EX_co2_e']
    df_ros.loc[_idx,'model_o2'] = me.solution.x_dict['EX_o2_e']
    df_ros.loc[_idx,'proteome_damage'] = _damage
    df_ros.loc[_idx,'resistance'] = _resistance
    df_ros.loc[_idx,'persistence'] = _persistence
    
    filename = 'html_files/ros_'+str(_condition)+'_scaling_'+str(_scaling)+'maintenance'
#     solutionfile=str(filename)+'.pickle'                                                        
#     with open(solutionfile,'wb') as f:
#         pickle.dump(me,f)
    htmlfile = str(filename) + '.html'
    show_escher_map(stress).save_html(htmlfile)

df_ros.head()

iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.338444 seconds
Finished makeME_LP in 1.010154 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 24.7694 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.320976 seconds
Finished makeME_LP in 1.012124 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 9.45513 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.345521 seconds
Finished makeME_LP in 1.009320 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.22653 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.370908 seconds
Finished makeME_LP in 1.014807 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.33062 seconds


,scaling,oxygen_uptake,glucose_uptake,model_o2,model_co2,proteome_damage,resistance,persistence
HMP3427_1,1,-61.0252,-1.38194,-3.53372,3.85864,0.00720109,0.000893062,0.00853298
HMP3427_5,5,-61.0252,-1.66924,-5.2516,5.57432,0.00822515,0.00124157,0.0115012
HMP3427_10,10,-61.0252,-2.02836,-7.39893,7.7189,0.00950526,0.00181225,0.015212
HMP3427_15,15,-61.0252,-2.38748,-9.54627,9.86348,0.0107856,0.00238302,0.0189234


In [58]:
# print(me.solution.x_dict.items())
# print(sorted(me.solution.x_dict))
# me.solution.x_dict['biomass_dilution']

_list = ['ATPM','EX_co2']
_dict = {key: me.solution.x_dict[key] for key in sorted(me.solution.x_dict)}
_matching_keys = []

for key, value in _dict.items():
    for substr in _list:
        if substr in key:
            _matching_keys.append(key)

for key in _matching_keys:
    print(f"Reaction: {key}, Flux: {_dict[key]}")

Reaction: ATPM, Flux: 0.1
Reaction: EX_co2_e, Flux: 12.105044751830759


In [61]:
# _list_scaling = [1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10,12,14,16,18,20,22,25]
_list_scaling = [1,5,10,20]
_base_condition = "WT"
_base_oxygen_uptake = df_model.loc[_base_condition,'oxygen_uptake']
_base_glucose_uptake = df_model.loc[_base_condition,'glucose_uptake']

df_ros = pd.DataFrame(columns=['scaling','oxygen_uptake','glucose_uptake','proteome_damage','resistance','persistence'])

for _idx in df_model.index:
    _condition = _idx
    _scaling = df_model.loc[_condition,'oxygen_uptake'] / _base_oxygen_uptake
    _oxygen_uptake = _scaling * _base_oxygen_uptake
    _glucose_uptake = df_model.loc[_condition,'glucose_uptake']
    _glucose_uptake = _base_glucose_uptake
    ros_dict = {'h2o2_c': 5e-08*_scaling, 'o2s_c': 2e-10*_scaling, 'h2o2_e': 1.5e-08*_scaling,
                    'o2s_e': 0, 'pq2_c': 0}
    
    MU_PREC = 1e-4
    MU_MIN  = df_model.loc[_condition,'growth_rate']
    MU_MAX  = df_model.loc[_condition,'growth_rate']
    
    rxn = me.reactions.EX_glc__D_e
    rxn.lower_bound = _glucose_uptake-0.10
    rxn.upper_bound = _glucose_uptake+0.10
    
    rxn = me.reactions.EX_for_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    
    rxn = me.reactions.ATPM
    rxn.lower_bound = _scaling
    
    # rxn = me.reactions.EX_co2_e
    # rxn.lower_bound = df_model.loc[_condition,'oxygen_uptake']-0.1
    # rxn.upper_bound = df_model.loc[_condition,'oxygen_uptake']+0.1
    
    # set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
    # set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
    # set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])
    
    stress.substitute_ros(solver, ros_dict)
    stress.substitute_metal(solver)

    sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX, basis = hs) #force growth rate

    hs = sol[1]
    
    _damage = get_sum_damage(me.solution)
    _resistance = get_sum(me.solution,iM_dict['resistance'])
    _persistence = get_sum(me.solution,iM_dict['persistence'])
    
    df_ros.loc[_idx,'scaling'] = _scaling
    df_ros.loc[_idx,'oxygen_uptake'] = _oxygen_uptake
    df_ros.loc[_idx,'glucose_uptake'] = _glucose_uptake
    df_ros.loc[_idx,'proteome_damage'] = _damage
    df_ros.loc[_idx,'resistance'] = _resistance
    df_ros.loc[_idx,'persistence'] = _persistence

for _scaling in _list_scaling:
    _condition = _base_condition
    _idx = str(_base_condition)+'_'+str(_scaling)
    _oxygen_uptake = _scaling * _base_oxygen_uptake
    # _do2_uptake = ((_o2 * _base_o2_uptake) - _base_o2_uptake)
    # _glucose_uptake = _base_glucose_uptake + (((_scaling * _base_oxygen_uptake) - _base_oxygen_uptake)/6)
    _glucose_uptake = _base_glucose_uptake
    ros_dict = {'h2o2_c': 5e-08*_scaling, 'o2s_c': 2e-10*_scaling, 'h2o2_e': 1.5e-08*_scaling,
                    'o2s_e': 0, 'pq2_c': 0}
    
    MU_PREC = 1e-4
    MU_MIN  = df_model.loc[_condition,'growth_rate']
    MU_MAX  = df_model.loc[_condition,'growth_rate']
    
    rxn = me.reactions.EX_glc__D_e
    rxn.lower_bound = _glucose_uptake-0.10
    rxn.upper_bound = _glucose_uptake+0.10
    
    rxn = me.reactions.EX_for_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000



    
    # set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
    # set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
    # set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])
    
    stress.substitute_ros(solver, ros_dict)
    stress.substitute_metal(solver)

    sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX, basis = hs) #force growth rate

    hs = sol[1]
    
    _damage = get_sum_damage(me.solution)
    _resistance = get_sum(me.solution,iM_dict['resistance'])
    _persistence = get_sum(me.solution,iM_dict['persistence'])
    
    # print(f'{_condition} Damage: {get_sum_damage(_damage)}')
    
    df_ros.loc[_idx,'scaling'] = _scaling
    df_ros.loc[_idx,'oxygen_uptake'] = _oxygen_uptake
    df_ros.loc[_idx,'glucose_uptake'] = _glucose_uptake
    df_ros.loc[_idx,'proteome_damage'] = _damage
    df_ros.loc[_idx,'resistance'] = _resistance
    df_ros.loc[_idx,'persistence'] = _persistence

df_ros.head()

iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.363354 seconds
Finished makeME_LP in 1.037188 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.36915 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.363717 seconds
Finished makeME_LP in 1.035086 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 8.75298 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.328173 seconds
Finished makeME_LP in 1.031013 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 19.3864 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.320758 seconds
Finished makeME_LP in 1.043657 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 10.7492 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished subst

,scaling,oxygen_uptake,glucose_uptake,proteome_damage,resistance,persistence
WT,1,-2.44101,-1.17982,0.00515149,0.00054553,0.00842116
HMP3427,2.5,-6.10252,-1.17982,0.0128713,0.000545314,0.0103399
WT_1,1,-2.44101,-1.17982,0.00870397,0.000392891,0.00759378
WT_5,5,-12.205,-1.17982,0.0503188,0.000352449,0.0144746
WT_10,10,-24.4101,-1.17982,0.0724475,0.00186065,0.0226474


In [105]:
print(me.reactions)

[<SummaryVariable biomass_dilution at 0x7f6d86a31c18>, <SummaryVariable protein_biomass_to_biomass at 0x7f6d892e4dd8>, <SummaryVariable mRNA_biomass_to_biomass at 0x7f6d892e4d30>, <SummaryVariable tRNA_biomass_to_biomass at 0x7f6d892e4cc0>, <SummaryVariable rRNA_biomass_to_biomass at 0x7f6d892e4c50>, <SummaryVariable ncRNA_biomass_to_biomass at 0x7f6d86a31358>, <SummaryVariable DNA_biomass_to_biomass at 0x7f6d86a31198>, <SummaryVariable lipid_biomass_to_biomass at 0x7f6d86a311d0>, <SummaryVariable constituent_biomass_to_biomass at 0x7f6d86a31160>, <SummaryVariable prosthetic_group_biomass_to_biomass at 0x7f6d86a310b8>, <SummaryVariable peptidoglycan_biomass_to_biomass at 0x7f6d86a31240>, <MEReaction DM_4CRSOL at 0x7f6d86a31588>, <MEReaction DM_5DRIB at 0x7f6d86a31320>, <MEReaction DM_AACALD at 0x7f6d86c21048>, <MEReaction DM_AMOB at 0x7f6d86c21c18>, <MEReaction DM_MTHTHF at 0x7f6d86c21748>, <MEReaction DM_OXAM at 0x7f6d86b0d4a8>, <MEReaction EX_12ppd__R_e at 0x7f6d86b0ddd8>, <MEReactio

In [ ]:
_list_scaling = [10,25]

for _scaling in _list_scaling:
    _condition = _base_condition
    _idx = str(_base_condition)+'_'+str(_scaling)
    _oxygen_uptake = _scaling * _base_oxygen_uptake
    # _do2_uptake = ((_o2 * _base_o2_uptake) - _base_o2_uptake)
    _glucose_uptake = _base_glucose_uptake + (((_scaling * _base_oxygen_uptake) - _base_oxygen_uptake)/6)
    ros_dict = {'h2o2_c': 5e-08*_scaling, 'o2s_c': 2e-10*_scaling, 'h2o2_e': 1.5e-06*_scaling,
                    'o2s_e': 0, 'pq2_c': 0}
    
    MU_PREC = 1e-4
    MU_MIN  = df_model.loc[_condition,'growth_rate']
    MU_MAX  = df_model.loc[_condition,'growth_rate']
    
    rxn = me.reactions.EX_glc__D_e
    rxn.lower_bound = _glucose_uptake-10.10
    rxn.upper_bound = _glucose_uptake+10.10
    
    rxn = me.reactions.EX_for_e
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000

    
    set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
    set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
    set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])
    
    stress.substitute_ros(solver, ros_dict)
    stress.substitute_metal(solver)

    sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX, basis = hs) #force growth rate

    hs = sol[1]
    
    _damage = get_sum_damage(me.solution)
    _resistance = get_sum(me.solution,iM_dict['resistance'])
    _persistence = get_sum(me.solution,iM_dict['persistence'])
    
    # print(f'{_condition} Damage: {get_sum_damage(_damage)}')
    
    df_ros.loc[_idx,'scaling'] = _scaling
    df_ros.loc[_idx,'oxygen_uptake'] = _oxygen_uptake
    df_ros.loc[_idx,'glucose_uptake'] = _glucose_uptake
    df_ros.loc[_idx,'proteome_damage'] = _damage
    df_ros.loc[_idx,'resistance'] = _resistance
    df_ros.loc[_idx,'persistence'] = _persistence

df_ros.head()

iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.559383 seconds
Finished makeME_LP in 1.035157 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 20.5486 seconds
iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.522534 seconds
Finished makeME_LP in 1.034315 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 9.55322 seconds


,scaling,oxygen_uptake,glucose_uptake,proteome_damage,resistance,persistence
WT,1,-2.44101,-1.17982,0.00673771,0.000393582,0.0079537
HMP3427,2.5,-6.10252,-1.83076,0.0162339,0.000350729,0.00941621
WT_1,1,-2.44101,-1.17982,0.00673771,0.000393582,0.0079537
WT_1.5,1.5,-3.66151,-1.38324,0.00946686,0.000381644,0.00817755
WT_2,2,-4.88202,-1.58666,0.0121151,0.000371128,0.00858819


In [ ]:
# get_sum_damage(me.solution)
# dir(stress.builder)

['SeqIO',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'add_complex_to_model',
 'add_dummy_reactions',
 'add_m_model_content',
 'add_metabolic_reaction_to_model',
 'add_model_complexes',
 'add_reactions_from_stoichiometric_data',
 'add_subreaction_data',
 'add_transcription_reaction',
 'add_translation_reaction',
 'build_reactions_from_genbank',
 'cobrame',
 'convert_aa_codes_and_add_charging',
 'create_transcribed_gene',
 'division',
 'dogma',
 'iteritems',
 'pandas',
 'print_function',
 'warn']

In [33]:
#basal first model for basis
ros_dict_conditions = {'WT': {'h2o2_c': 5e-08, 'o2s_c': 2e-10, 'h2o2_e': 1.5e-06,
            'o2s_e': 0, 'pq2_c': 0},
            'HMP3427': {'h2o2_c': 1.25e-07, 'o2s_c': 5e-10, 'h2o2_e': 1.5e-06,
            'o2s_e': 0, 'pq2_c': 0}}

In [ ]:
# Model ROS Flux as a multiple of O2 uptake
_base_condition = "WT"
for _condition in df_model.index:
    scaling_factor = (df_model.loc[_condition,'oxygen_uptake']/ 1) / (df_model.loc[_base_condition,'oxygen_uptake']/ 1)
    ros_dict = {'h2o2_c': 5e-08*i, 'o2s_c': 2e-10*i, 'h2o2_e': 1.5e-06*i,
                    'o2s_e': 0, 'pq2_c': 0}

In [ ]:
# Set Strain
_condition_list = df_model.index
_condition = _condition_list[0]

# # growth media conditions                                                                                                  CHECK

MU_PREC = 1e-4
MU_MIN  = df_model.loc[_condition,'growth_rate']
MU_MAX  = df_model.loc[_condition,'growth_rate']

# Flux conditions

rxn = me.reactions.EX_glc__D_e
rxn.lower_bound = df_model.loc[_condition,'glucose_uptake']-0.10
rxn.upper_bound = df_model.loc[_condition,'glucose_uptake']+0.10
# print(me.reactions.EX_glc__D_e.bounds)

# rxn = me.reactions.EX_o2_e
# rxn.lower_bound = df_model.loc[_condition,'oxygen_uptake']-0.010
# rxn.upper_bound = df_model.loc[_condition,'oxygen_uptake']+0.010
# print(me.reactions.EX_o2_e.bounds)

# rxn = me.reactions.?mel
# rxn.lower_bound = df_model.loc[_condition,'melatonin_secretion']
# rxn.upper_bound = df_model.loc[_condition,'melatonin_secretion']

rxn = me.reactions.EX_co2_e
rxn.lower_bound = df_model.loc[_condition,'oxygen_uptake']-1
rxn.upper_bound = df_model.loc[_condition,'oxygen_uptake']+1

rxn = me.reactions.EX_ac_e
rxn.lower_bound = -1000 #df_model.loc[_condition,'acetate_secretion']
rxn.upper_bound = 1000 #df_model.loc[_condition,'acetate_secretion']

rxn = me.reactions.EX_for_e
rxn.lower_bound = df_model.loc[_condition,'formate_secretion']
rxn.upper_bound = df_model.loc[_condition,'formate_secretion']

rxn = me.reactions.EX_lac__D_e
rxn.lower_bound = df_model.loc[_condition,'lactate_secretion']
rxn.upper_bound = df_model.loc[_condition,'lactate_secretion']

rxn = me.reactions.EX_pyr_e
rxn.lower_bound = df_model.loc[_condition,'pyruvate_secretion']
rxn.upper_bound = df_model.loc[_condition,'pyruvate_secretion']

rxn = me.reactions.EX_etoh_e
rxn.lower_bound = df_model.loc[_condition,'ethanol_secretion']
rxn.upper_bound = df_model.loc[_condition,'ethanol_secretion']

# Check
rxn = me.reactions.EX_for_e
rxn.lower_bound = -1000
rxn.upper_bound = 1000


set_expression_ratio(me,'pathway_TCA','pathway_rproteins',df_model.loc[_condition,'TCA_mf']/df_model.loc[_condition,'rprotein_mf'])
set_expression_ratio(me,'pathway_glycolysis','pathway_rproteins',df_model.loc[_condition,'glycolysis_mf']/df_model.loc[_condition,'rprotein_mf']) 
set_expression_ratio(me,'pathway_PPP','pathway_rproteins',df_model.loc[_condition,'PPP_mf']/df_model.loc[_condition,'rprotein_mf'])

ros_dict = ros_dict_conditions[_condition]

In [41]:
# #basal first model for basis
# ros_dict = {'h2o2_c': 5e-08, 'o2s_c': 2e-10, 'h2o2_e': 1.5e-06,
#             'o2s_e': 0, 'pq2_c': 0}
stress.substitute_ros(solver, ros_dict)
stress.substitute_metal(solver)
sol = solver.bisectmu(precision=MU_PREC, mumin=MU_MIN, mumax=MU_MAX) #constrain with Mu
# sol = solver.bisectmu(precision=MU_PREC) #constrain without Mu

iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 5.416517 seconds
Finished makeME_LP in 1.010205 seconds
Getting MINOS parameters from ME_NLP...
1 0.1 0.1 0.1 0.1 optimal
Bisection done in 190.163 seconds


In [ ]:
# get_sum_damage(me.solution)
# dir(stress.builder)

['SeqIO',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'add_complex_to_model',
 'add_dummy_reactions',
 'add_m_model_content',
 'add_metabolic_reaction_to_model',
 'add_model_complexes',
 'add_reactions_from_stoichiometric_data',
 'add_subreaction_data',
 'add_transcription_reaction',
 'add_translation_reaction',
 'build_reactions_from_genbank',
 'cobrame',
 'convert_aa_codes_and_add_charging',
 'create_transcribed_gene',
 'division',
 'dogma',
 'iteritems',
 'pandas',
 'print_function',
 'warn']

In [115]:
# #knock out BW25113 genes

# KO_genes = ['b3126', 'b3127', 'b3128', 'b0344', 'b1241', 'b1251', 'b1252']

# for i in KO_genes:
#     try:
#         me.reactions.get_by_id('translation_' + str(i)).knock_out()
#     except:
#         pass
# print('genes knocked out')

genes knocked out


In [48]:
class SpontROS(object):
    """
    Class for adding spontaneous ROS generation.
    Port of JM Monk's add_ros.py to ME models.
    """
    def __init__(self, me, ros_coeff_dict={'h2o2_c':'c', 'o2s_c':'k'}):
            
        self.me = me
        self.ros_coeff_dict = ros_coeff_dict
        # Placeholder for spont ros rxn list
        self.ros_rxns = []
        self.ros_stoichs = []   # ditto for ros stoichiometric_data
        self.stoich_coeff_dict = {}    # map rxn to c & k coeffs

    def pathto(self, filename):
        return os.path.join(oxidizeme.__path__[0], 'data', filename)

    
    def add_ros_rxns(self, rxn_file='spont_ros_rxns.csv', #spont_ros_rxns_me2.csv
            col_name='Rxn Name',
            col_h2o2='Putative h2o2 Rxn', col_o2s='Putative o2s Rxn',
            met_repl_dict={'[c]':'_c', '[p]':'_p',
                '-L':'__L', '-D':'__D', '-S':'__S'},
            complex_file='protein_complexes.txt',
            verbosity=1):
        """
        add_ros_rxns(self, in_model, cvals, kvals, name='')

        Adds spontaneous ROS generation to stoichiometric_data
        stoichiometry for each of Brynildsen's ROS-generating reactions.
        """
        me = self.me
        ros_coeff_dict = self.ros_coeff_dict
        sym_h2o2 = ros_coeff_dict['h2o2_c']
        sym_o2s  = ros_coeff_dict['o2s_c']
    
        # Temp model object that holds all the ROS reactions
        mdl = Model('ros')

        # Exclude compartment from complexes
        # df_cplx = pd.read_csv(self.pathto(complex_file), sep='\t', header=None)
        # complexes = df_cplx[0].tolist()

        # Load Brynildsen's reactions
        # df_ros = pd.read_csv(self.pathto(rxn_file)).dropna(subset=[col_h2o2, col_o2s])
        df_ros = pd.read_csv(rxn_file).dropna(subset=[col_h2o2, col_o2s])

        stoichs_mod = []

        for irow, row in df_ros.iterrows():
            rxn_str = row[col_h2o2]
            rxn_name = row[col_name]
            if not me.stoichiometric_data.has_id(rxn_name):
                print(rxn_name, 'not in stoichiometric_data. Skipping.')
                continue

            stoich = me.stoichiometric_data.get_by_id(rxn_name)
            stoichs_mod.append(stoich)
            
            rxn_h2o2 = Reaction(stoich.id+'_h2o2')
            rxn_o2s  = Reaction(stoich.id+'_o2s')
            mdl.add_reactions([rxn_h2o2, rxn_o2s])

            # Get h2o2 and o2s rxns
            rxn_strs = row['Putative h2o2 Rxn']
            # Sometimes the reaction string has multiple rxns separated by commas
            for rxn_str in rxn_strs.split(','):
                rxn_h2o2.build_reaction_from_string(rxn_str, verbose=False)

            rxn_strs = row['Putative o2s Rxn']
            for rxn_str in rxn_strs.split(','):
                rxn_o2s.build_reaction_from_string(rxn_str, verbose=False)

            # Keep c & k stoichiometries for h2o2 and o2s as Symbols,
            # substituted at the time of simulation, like mu.
            # Not too many so probably don't need to compile expressions like we do for mu
            # Note: different c & k for each rxn
            s_h2o2 = Symbol(sym_h2o2+'_'+str(irow), positive=True)
            s_o2s  = Symbol(sym_o2s+'_'+str(irow), positive=True)
            for met, s in rxn_h2o2.metabolites.items():
                rxn_h2o2._metabolites[met] = s * s_h2o2

            for met, s in rxn_o2s.metabolites.items():
                rxn_o2s._metabolites[met] = s * s_o2s

            # Fix compartments and other ID inconsistencies
            for met in rxn_h2o2._metabolites.keys():
                for old, new in met_repl_dict.items():
                    met.id = met.id.replace(old, new)
                    # No compartment for complexes
                    id2 = replace_last(met.id, new, '')
                    id3 = id2.split('_mod')[0]  # modified complex not in complex list
                    # if id3 in complexes:
                    #     met.id = id2

            for met in rxn_o2s._metabolites.keys():
                for old, new in met_repl_dict.items():
                    met.id = met.id.replace(old, new)
                    # No compartment for complexes
                    id2 = replace_last(met.id, new, '')
                    id3 = id2.split('_mod')[0]  # modified complex not in complex list
                    # if id3 in complexes:
                    #     met.id = id2

            #------------------------------------------------
            # Ensure that metabolite IDs in ros rxns consistent with ME
            # If not, stop now.
            for met in rxn_h2o2.metabolites:
                if not me.metabolites.has_id(met.id):
                    raise Exception(met.id+' not in me model. Check that ids are consistent.')

            for met in rxn_o2s.metabolites:
                if not me.metabolites.has_id(met.id):
                    raise Exception(met.id+' not in me model. Check that ids are consistent.')
            #------------------------------------------------

            # Add the original (pre-ROS) stoichiometry to ROS portion
            # Option 1: alter the stoichiometric_data and push to parent_reactions
            # Option 2: add stoichiometry to parent_reactions
            for rxn in stoich.parent_reactions:
                # Make sure that ROS only generated, not consumed spontaneously
                if rxn.reverse:
                    if verbosity > 0:
                        print('Skipping reverse rxn:', rxn.id)

                else:
                    rxn.add_metabolites(rxn_h2o2.metabolites)
                    rxn.add_metabolites(rxn_o2s.metabolites)
                    self.ros_rxns.append(rxn)
                    self.stoich_coeff_dict[stoich.id] = {'ind': irow, 'h2o2_c': s_h2o2, 'o2s_c': s_o2s}

In [49]:
rxn_file='spont_ros_rxns.csv'
col_h2o2='Putative h2o2 Rxn'
col_o2s='Putative o2s Rxn'
df_ros = pd.read_csv(rxn_file).dropna(subset=[col_h2o2, col_o2s])
df_ros

,Rxn Name,Flavin,Quinone,Fe-S,Heme,Reversible,iAF1260 index,Rxn (iAF1260),Putative h2o2 Rxn,Putative o2s Rxn,"Combined rxn (c -> h2o2 rxn, k->o2s rxn)",Notes
0,ACHBS,1.0,0.0,0.0,0.0,0.0,166.0,[c] : 2obut + h + pyr --> 2ahbut + co2,[c] : pyr + o2 + h2o ---> ac + co2 + h2o2,[c] : pyr + (2) o2 + h2o ---> ac + co2 + (2) ...,[c] : 2obut + h + (1+c+k) pyr + (c+2k) o2 + (c...,PMID: 15236573
1,ACLS,1.0,0.0,0.0,0.0,0.0,168.0,[c] : h + (2) pyr --> alac-S + co2,[c] : pyr + o2 + h2o ---> ac + co2 + h2o2,[c] : pyr + (2) o2 + h2o ---> ac + co2 + (2) ...,[c] : h + (2+c+k) pyr + (c+2k) o2 + (c+k) h2o ...,PMID: 15236573
2,ACOAD1f,1.0,0.0,0.0,0.0,1.0,178.0,[c] : btcoa + fad <==> b2coa + fadh2,[c] : o2 + fadh2 --> fad + h2o2,[c] : (2) o2 + fadh2 --> fad + (2) o2s + (2) h,[c] : btcoa + (1-c-k) fad + (c+2k) o2 --> b2co...,"PMID: 14527285, General Rule: reduced flavin d..."
3,ACOAD2f,1.0,0.0,0.0,0.0,1.0,179.0,[c] : fad + hxcoa <==> fadh2 + hx2coa,[c] : o2 + fadh2 --> fad + h2o2,[c] : (2) o2 + fadh2 --> fad + (2) o2s + (2) h,[c] : hxcoa + (1-c-k) fad + (c+2k) o2 --> hx2c...,"PMID: 14527285, General Rule: reduced flavin d..."
4,ACOAD3f,1.0,0.0,0.0,0.0,1.0,180.0,[c] : fad + occoa <==> fadh2 + oc2coa,[c] : o2 + fadh2 --> fad + h2o2,[c] : (2) o2 + fadh2 --> fad + (2) o2s + (2) h,[c] : occoa + (1-c-k) fad + (c+2k) o2 --> oc2c...,"PMID: 14527285, General Rule: reduced flavin d..."
5,ACOAD4f,1.0,0.0,0.0,0.0,1.0,181.0,[c] : dcacoa + fad <==> dc2coa + fadh2,[c] : o2 + fadh2 --> fad + h2o2,[c] : (2) o2 + fadh2 --> fad + (2) o2s + (2) h,[c] : dcacoa + (1-c-k) fad + (c+2k)o2 --> dc2c...,"PMID: 14527285, General Rule: reduced flavin d..."
6,ACOAD5f,1.0,0.0,0.0,0.0,1.0,182.0,[c] : ddcacoa + fad <==> dd2coa + fadh2,[c] : o2 + fadh2 --> fad + h2o2,[c] : (2) o2 + fadh2 --> fad + (2) o2s + (2) h,[c] : ddcacoa + (1-c-k) fad + (c+2k) o2 --> dd...,"PMID: 14527285, General Rule: reduced flavin d..."
7,ACOAD6f,1.0,0.0,0.0,0.0,1.0,183.0,[c] : fad + tdcoa <==> fadh2 + td2coa,[c] : o2 + fadh2 --> fad + h2o2,[c] : (2) o2 + fadh2 --> fad + (2) o2s + (2) h,[c] : tdcoa + (1-c-k) fad + (c+2k) o2 --> td2c...,"PMID: 14527285, General Rule: reduced flavin d..."
8,ACOAD7f,1.0,0.0,0.0,0.0,1.0,184.0,[c] : fad + pmtcoa <==> fadh2 + hdd2coa,[c] : o2 + fadh2 --> fad + h2o2,[c] : (2) o2 + fadh2 --> fad + (2) o2s + (2) h,[c] : pmtcoa + (1-c-k) fad + (c+2k) o2 --> hdd...,"PMID: 14527285,General Rule: reduced flavin do..."
9,ACOAD8f,1.0,0.0,0.0,0.0,1.0,185.0,[c] : fad + stcoa <==> fadh2 + od2coa,[c] : o2 + fadh2 --> fad + h2o2,[c] : (2) o2 + fadh2 --> fad + (2) o2s + (2) h,[c] : stcoa + (1-c-k) fad + (c+2k) o2 --> od2c...,"PMID: 14527285, General Rule: reduced flavin d..."


In [50]:
spont = SpontROS(solver)

In [ ]:
spont.add_ros_rxns()

In [52]:
me.stoichiometric_data

<generator object stoichiometric_data at 0x7fcc8466c7d8>

In [ ]:
# print(me.solution.x_dict.items())
# print(sorted(me.solution.x_dict))
# me.solution.x_dict['biomass_dilution']

_list = ['ATPM','EX_co2','DNA','Fenton_FWD_SPONT']
_dict = {key: me.solution.x_dict[key] for key in sorted(me.solution.x_dict)}
_matching_keys = []

for key, value in _dict.items():
    for substr in _list:
        if substr in key:
            _matching_keys.append(key)

for key in _matching_keys:
    print(f"Reaction: {key}, Flux: {_dict[key]}")

Reaction: ATPM, Flux: 1.0
Reaction: DNA_biomass_to_biomass, Flux: 0.005927308655337385
Reaction: DNA_damage, Flux: 0.0
Reaction: DNA_replication, Flux: 0.019248080930467183
Reaction: EX_co2_e, Flux: 2.641008950366152
Reaction: Fenton_FWD_SPONT, Flux: 1.3197098927966787e-31
